<a id='top'></a>
# Table of contents
- [Random Generator](#randomgenerator)
- [Environment](#environment)
- [Infrastructure Object](#infrastructureobject)
- [Processing Station](#processingstation)
- [Machine](#machine)
- [Buffer](#buffer)
- [Source](#source)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)

# To-Do:
- [ ] add machine groups (parallel machines)
    - logic behind communicating machine groups
    - registration of machines in groups
- [ ] machine groups: add different allocation rules
    - lowest occupation time
- [ ] capacity deadlocks between buffers and machines
    - [ ] resolve necessary! otherwise no information about blockages
    - [x] ~~remove deadlocks by counting the associated machines and comparing the counter to the buffer's capacity~~ (**no capacities > 1 for processing stations**)
    - **problem persists: if predecessor systems produce faster than the target processing stations, they fill the buffer until it is filled --> result == deadlock again if buffer is shared between processing station**
    - shared buffers: if only used for machine groups --> no problem because there are no circles between these machines
    - [ ] look into problems where processing stations have a capacity greater than 1
- [ ] add feasibility check
    - since agents can choose actions which are not feasible a feasiblity check is necessary
    - describe learning flow if a non-feasible action was chosen
- [ ] add logistic objective values:
    - WIP
    - [x] ~~lead time~~
    - [x] ~~utilisation~~
- [ ] add Gantt chart visualisation for debugging
    - [x] ~~after simulation run~~
    - [ ] during simulation run
- [ ] logic/ interface for generation of multiple jobs
    - using dispatcher or source?
    - interface: design + properties
- [ ] add priority rules
    - currently: ``FIFO, LIFO, SPT, LPT``
- [ ] tracking disjunctive graph model
    - **[QUESTION] Working with parallel machines?**
- [ ] initialisation of the model with pre-defined state information
    - [x] ~~build base by implementing a pre-process method (initialisation) before the simulation starts~~
    - [x] ~~initialise/create objects with pre-defined states~~
- [ ] add machine breakdowns
    - vide **[Check Interruption](#MachineBreakdownTest)**

- [x] ~~description of process logic for generation and entry of jobs~~
- [x] ~~add setting of job/machine states in machine logic~~
    - using state update function which combine all necessary state update calls
- [x] ~~add status info to job DB? (waiting, processing, ...) (disposable = waiting?)~~
    - advantage:
        - one central DB with all information, no cluttered information
        - simple filtering for jobs by current status incl. disposable jobs
- [x] ~~add sources and sinks~~
    - [x] source: generates new job with given intervals
    - [x] sink: destroys jobs and finalises data collection
- [x] ~~add physical buffers~~

- [x] ~~add bar charts for time components~~
- [x] ~~add monitor class to unitise data collection~~
- [x] ~~I/O functions for elements~~

- [ ] self-marking as disposable by jobs
    - ==*check if still necessary*==
    - includes demarking
    - currently only addition implemented
- [x] ~~register job object in dispatcher~~
- [x] ~~register operations in dispatcher~~
- [x] ~~add uniqueness check for custom IDs in resource objects (only interface to user)~~
- [x] ~~operations list of jobs as deque~~
- [x] ~~add generic infrastructure class from which infrastructure objects are derived~~
- [x] ~~implement routing logic in objects~~
- [x] ~~add operation starting and end points~~

### Logic behind simulation model approach
*Model of Resource and Load*:
- system consists of physical objects, also called infrastructure
    - each element can be considered as encapsulated resource
- stress can be put on each system by occupying resources, also called load
    - definition of load depends on system type and modelling, e.g. production jobs for production systems or customers for cashiers in a shop
- load objects are called ***load unit***

*Guiding Priciples*:
- **load objects** can only be spatially and temporally modified by **resources**
    - only resource objects can put load objects on other resources and change their state
    - load objects **contain the necessary information** which is essential for their further processing

################

Logic of ``Lang et al.: Modeling Production Scheduling Problems as Reinforcement Learning Environments based on Discrete-Event Simulation and OpenAI Gym``
- whole routing logic is implemented in a collaborative manner between resources and load objects
    - each load object puts itself in a associated queue
    - therefore load objects can change their *states* and *location* by theirown
- **violates resource-load model: no load object can change its state without a associated resource**

In [713]:
from __future__ import annotations
import numpy as np
import numpy.typing as npt
from numpy.random._generator import Generator
import random
import simpy
import salabim as sim
from salabim import Queue, State
from typing import TypeAlias, Iterable, Iterator, Any
from collections import OrderedDict, deque
from operator import attrgetter
from functools import lru_cache
import logging
import sys
import pandas as pd
from pandas import DataFrame, Series
import plotly.express as px
from plotly.graph_objs._figure import Figure

sim.yieldless(False)

# type aliases
NPRandomGenerator: TypeAlias = Generator
SimPyEnv: TypeAlias = simpy.core.Environment
SalabimEnv: TypeAlias = sim.Environment
EnvID: TypeAlias = int
#JobID: TypeAlias = int
#OpID: TypeAlias = int
ObjectID: TypeAlias = int
### [CHANGE] Replace MachineID as CustomID
MachineID: TypeAlias = int | str
CustomID: TypeAlias = int | str
#InfstructObj: TypeAlias = object # better naming in future
PlotlyFigure: TypeAlias = Figure

# forward reference, referenced before assignment
#Job: TypeAlias = 'Job'
#Dispatcher: TypeAlias = 'Dispatcher'

# logging
# IPython compatibility
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
LOGGING_LEVEL = 'DEBUG'
LOGGING_LEVEL_ENV = 'ERROR'
LOGGING_LEVEL_DISPATCHER = 'DEBUG'
LOGGING_LEVEL_SOURCES = 'ERROR'
LOGGING_LEVEL_SINKS = 'ERROR'
LOGGING_LEVEL_PRODSTATIONS = 'ERROR'
LOGGING_LEVEL_JOBS = 'ERROR'
LOGGING_LEVEL_OPERATIONS = 'ERROR'
LOGGING_LEVEL_BUFFERS = 'ERROR'
LOGGING_LEVEL_MONITORS = 'ERROR'


logger = logging.getLogger('base')
logger.setLevel(LOGGING_LEVEL)
logger_env = logging.getLogger('env')
logger_env.setLevel(LOGGING_LEVEL_ENV)
logger_dispatcher = logging.getLogger('dispatcher')
logger_dispatcher.setLevel(LOGGING_LEVEL_DISPATCHER)
logger_sources = logging.getLogger('sources')
logger_sources.setLevel(LOGGING_LEVEL_SOURCES)
logger_sinks = logging.getLogger('sinks')
logger_sinks.setLevel(LOGGING_LEVEL_SINKS)
logger_prodStations = logging.getLogger('prodStations')
logger_prodStations.setLevel(LOGGING_LEVEL_PRODSTATIONS)
logger_buffers = logging.getLogger('buffers')
logger_buffers.setLevel(LOGGING_LEVEL_BUFFERS)
logger_monitors = logging.getLogger('monitors')
logger_monitors.setLevel(LOGGING_LEVEL_MONITORS)

logger_jobs = logging.getLogger('jobs')
logger_jobs.setLevel(LOGGING_LEVEL_JOBS)
logger_operations = logging.getLogger('operations')
logger_operations.setLevel(LOGGING_LEVEL_OPERATIONS)



INF = float('inf')
FAIL_DELAY = 20 # time after a store request is failed

<a id='randomgenerator'></a>

In [714]:
class RandomJobGenerator(object):
    
    def __init__(
        self,
        seed: int = 42,
    ) -> None:
        """
        seed: seed value for random number generator
        """
        self._np_rnd_gen: NPRandomGenerator = np.random.default_rng(seed=seed)
        random.seed(seed)
        
    def gen_rnd_JSSP_inst(
        self,
        n_jobs: int,
        n_machines: int,
    ) -> tuple[npt.NDArray[np.uint16], npt.NDArray[np.uint16]]:
        """
        Generates random job shop instance with given number of and machines
        - each job on all machines
        - max processing time = 9
        
        Output:
        n_jobs: number of jobs
        n_machines: number of machines
        n_tasks: number of tasks
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
        # generate random process time matrix shape=(n_jobs, n_machines)
        mat_ProcTimes = self._np_rnd_gen.integers(1, 10, size=(n_jobs,n_machines), dtype=np.uint16)
        
        # generate randomly shuffled job machine combinations
        # machine IDs from 1 to n_machines
        temp = np.arange(0, n_machines, step=1, dtype=np.uint16)
        temp = np.expand_dims(temp, axis=0)
        # repeat dummy line until number n_jobs is reached
        temp = np.repeat(temp, n_jobs, axis=0)
        # randomly permute the machine indices job-wise
        mat_JobMachID = self._np_rnd_gen.permuted(temp, axis=1)
        
        # generate operation ID matrix
        # not mandatory because operations are registered in the environment's dispatcher
        n_ops = n_jobs * n_machines
        temp2 = np.arange(0, (n_ops), step=1, dtype=np.uint16)
        mat_OpID = temp2.reshape(n_jobs, -1)
        
        return mat_ProcTimes, mat_JobMachID
    
    def gen_rnd_job(
        self,
        n_machines: int,
    ) -> tuple[npt.NDArray[np.uint16], npt.NDArray[np.uint16]]:
        """generates random job with machine IDs
        [OUTDATED] Should be replaced by the more generic 'gen_rnd_job_by_ids' method
        which uses any IDs provided as NumPy array

        Parameters
        ----------
        n_machines : int
            _description_

        Returns
        -------
        tuple[npt.NDArray[np.uint16], npt.NDArray[np.uint16]]
            _description_
        """
        
        # generate random process time matrix shape=(n_machines)
        mat_ProcTimes = self._np_rnd_gen.integers(1, 10, size=n_machines, dtype=np.uint16)
        
        # generate randomly shuffled job machine combinations
        # machine IDs from 1 to n_machines
        temp = np.arange(0, n_machines, step=1, dtype=np.uint16)
        # randomly permute the machine indices job-wise
        mat_JobMachID = self._np_rnd_gen.permuted(temp)
        
        return mat_ProcTimes, mat_JobMachID
    
    def gen_rnd_job_by_ids(
        self,
        ids: list[CustomID],
        min_proc_time: int = 1,
        max_proc_time: int = 10,
    ) -> tuple[list[int], list[CustomID]]:
        """Generic function to generate processing times and execution flow of a job object
        - permute the given NumPy array of IDs with shape(1,n) along the columns
        - generate random processing times in the range of ``min_proc_time`` to ``max_proc_time``

        Parameters
        ----------
        ids : npt.NDArray[np.uint16]
            _description_
        min_proc_time : int, optional
            _description_, by default 1
        max_proc_time : int, optional
            _description_, by default 10

        Returns
        -------
        tuple[npt.NDArray[np.uint16], npt.NDArray[np.uint16]]
            _description_
        """
        n_objects: int = len(ids)
        
        # generate random process time matrix shape=(n_objects)
        #mat_ProcTimes = random.choices(range(min_proc_time, (max_proc_time+1)), k=n_objects)
        
        mat_ProcTimes = self._np_rnd_gen.integers(
                                            min_proc_time, 
                                            max_proc_time, 
                                            size=n_objects, 
                                            dtype=np.uint16).tolist()
        
        
        # randomly permute the object indices
        mat_JobMachID = self._np_rnd_gen.permuted(ids).tolist()
        #mat_JobMachID = ids.copy()
        #random.shuffle(mat_JobMachID)
        
        return mat_ProcTimes, mat_JobMachID

In [715]:
job_generator = RandomJobGenerator(seed=42)

In [716]:
job_generator.gen_rnd_job_by_ids(ids=['cust01', 'cust02', 'cust03'])

([2, 1, 9], ['cust01', 'cust03', 'cust02'])

In [717]:
mat_ProcTimes, mat_JobMachID = job_generator.gen_rnd_JSSP_inst(2,3)

In [718]:
mat_ProcTimes

array([[1, 4, 1],
       [8, 5, 1]], dtype=uint16)

In [719]:
mat_JobMachID

array([[0, 1, 2],
       [2, 0, 1]], dtype=uint16)

##### Brainstorming of ways to import machine names and IDs
*Case 1: only names are given*
- read machine names and assert IDs to them
- build data structure with name and ID bundled (maybe as property of a machine class)

*Case 2: IDs are given*
- only building data structure with name and ID bundled

*Data Structure:*
- if only mapping of two pairs in each direction (lookup ID or lookup machine name)
    - bi-directional dictionary

**for resource objects:**
- CustomID may not be of type 'None' because the custom identifiers are the only interface to the end user (EnvID are solely handled inernally)
- add checking for uniqueness of custom identifiers necessary, else the mapping of different objects could be ambiguous
- jobs and operations can use ambiguous custom IDs --> custom IDs can still be None

- dedicated environment class with information on associated resources and jobs
- maybe add possibility of using subsystems (bundle of resources with unique identifiers)

<a id='environment'></a>
**Salabim Env**

In [720]:
class SimulationEnvironment(sim.Environment):
    
    def __init__(
        self,
        **kwargs,
    ) -> None:
        """
        
        """
        super().__init__(**kwargs)
        
        # resource database as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'env_id': int,
            'custom_id': object,
            'resource': object,
            'name': str,
            'res_type': str,
            'state': str,
        }
        self._res_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._res_db = self._res_db.astype(self._infstruct_prop)
        self._res_db = self._res_db.set_index('env_id')
        self._res_lookup_props: set[str] = set(['env_id', 'custom_id', 'name'])
        
        # station group database as simple Pandas DataFrame
        self._station_group_prop: dict[str, type] = {
            'station_group_id': int,
            'custom_group_id': object,
            'station_group_name': str,
            'station_group': object,
        }
        self._station_group_db: DataFrame = pd.DataFrame(columns=list(self._station_group_prop.keys()))
        self._station_group_db = self._station_group_db.astype(self._station_group_prop)
        self._station_group_db = self._station_group_db.set_index('station_group_id')
        self._station_group_lookup_props: set[str] = set(['station_group_id', 'custom_group_id', 'station_group_name'])
        # station group identifiers
        self._station_group_counter: ObjectID = 0
        self._station_groups_custom_identifiers: set[CustomID] = set()
        
        # env identifiers
        self._id_counter: EnvID = 0
        self._res_custom_identifiers: set[CustomID] = set()
        
        # job dispatcher
        self._dispatcher_registered: bool = False
        self._dispatcher: Dispatcher = None
        
        # sink: pool of sinks possible to allow multiple sinks in one environment
        # [PERHAPS CHANGED LATER] 
        # currently only one sink out of the pool is chosen because jobs do not contain 
        # information about a target sink
        self._sink_registered: bool = False
        self._sinks: set[Sink] = set()
        
        # counter for processing stations (machines, assembly, etc.)
        self.num_proc_stations: int = 0
        
        return None
    
    def _obtain_env_id(self) -> EnvID:
        """Simple counter function for managing environment IDs"""
        # assign id and set counter up
        env_id = self._id_counter
        self._id_counter += 1
        
        return env_id
    
    def _obtain_station_group_id(self) -> ObjectID:
        """Simple counter function for managing station group IDs

        Returns
        -------
        ObjectID
            unique station group ID
        """
        st_group_id = self._station_group_counter
        self._station_group_counter += 1
        
        return st_group_id
    
    def register_dispatcher(
        self,
        dispatcher: Dispatcher,
    ) -> EnvID:
        """
        Registers a dispatcher instance for the environment. Only one instance per environment is allowed.
        returns: EnvID for the dispatcher instance
        """
        # obtain env_id
        env_id = self._obtain_env_id()
        
        if not self._dispatcher_registered:
            self._dispatcher = dispatcher
            self._dispatcher_registered = True
            logger_env.info(f"Successfully registered dispatcher with EnvID {env_id}")
        else:
            raise AssertionError("There is already a registered dispatcher instance \
                                 Only one instance per environement is allowed.")
        
        return env_id
    
    @property
    def dispatcher(self) -> Dispatcher:
        """obtain the current registered dispatcher instance of the environment"""
        if self._dispatcher is None:
            raise ValueError("No Dipsatcher instance registered.")
        else:
            return self._dispatcher
    
    def register_resource(
        self,
        obj: InfrastructureObject,
        custom_identifier: CustomID,
        name: str | None,
        state: str,
    ) ->  tuple[EnvID, str]:
        """
        registers an infrastructure object in the environment by assigning an unique id and 
        adding the object to the associated resources of the environment
        
        obj: env resource = instance of a subclass of InfrastructureObject
        custom_identifier: user defined identifier
        name: custom name of the object, \
            default: None
        returns:
            env_id: assigned env ID
        """
        # check for uniqueness of custom_identifier
        # type security
        if not isinstance(custom_identifier, (str, int)):
            raise TypeError("Custom identifier must be of type STR or INT")
        # create check value
        if isinstance(custom_identifier, str):
            # remove capital letters for checking
            check_val = custom_identifier.lower()
        else:
            check_val = custom_identifier
        
        # check if value already exists
        if check_val in self._res_custom_identifiers:
            raise ValueError(f"The custom identifier {custom_identifier} provided already exists, \
                            but has to be unique.")
        else:
            self._res_custom_identifiers.add(check_val)
        
        # obtain env_id
        env_id = self._obtain_env_id()
        
        # register sinks
        if isinstance(obj, Sink):
            if not self._sink_registered:
                self._sink_registered = True
            self._sinks.add(obj)
        
        # count number of machines
        if isinstance(obj, ProcessingStation):
            self.num_proc_stations += 1
        
        # custom name
        if name is None:
            name = f'{type(obj).__name__}_env_{env_id}'
        
        # new entry for resource data base
        new_entry: DataFrame = pd.DataFrame({
                                'env_id': [env_id],
                                'custom_id': [custom_identifier],
                                'resource': [obj],
                                'name': [name],
                                'res_type': [obj.res_type],
                                'state': [state]})
        new_entry = new_entry.astype(self._infstruct_prop)
        new_entry = new_entry.set_index('env_id')
        self._res_db = pd.concat([self._res_db, new_entry])
        
        logger_env.info(f"Successfully registered object with EnvID {env_id} and name {name}")
        
        return env_id, name
    
    def register_station_group(
        self,
        processing_station: ProcessingStation,
        custom_group_id: CustomID | None,
        station_group_name: str | None,
    ) -> tuple[ObjectID, StationGroup]:
        """registers an processing station in the corresponding station group

        Parameters
        ----------
        processing_station : ProcessingStation
            _description_
        station_group_identifier : CustomID | None
            _description_

        Returns
        -------
        StationGroup
            _description_
        """
        # type security
        # object to be registered
        """
        if isinstance(processing_station, ProcessingStation):
            raise TypeError(f"{processing_station} is not of type >>ProcessingStation<<, but it has to be.")
        """
        # custom identifier
        if custom_group_id is None:
            # assign station group name by the corresponding processing station
            custom_group_id = f'station_group_{processing_station.name()}'
        elif not isinstance(custom_group_id, (str, int)):
            raise TypeError("Provided custom identifier for station group must be of type STR or INT")
        
        # create check value
        if isinstance(custom_group_id, str):
            # remove capital letters for checking
            check_val = custom_group_id.lower()
        else:
            check_val = custom_group_id
        
        # check for uniqueness of custom_identifier
        if check_val in self._station_groups_custom_identifiers:
            # already exists, add to this station group
            # lookup in station group DB
            station_group = self.get_station_group_by_prop(
                                                    val=custom_group_id, 
                                                    property='custom_group_id')
            station_group_id = station_group.group_id
            # add proc station to associated ones
            station_group.add_processing_station(processing_station=processing_station)
        else:
            self._station_groups_custom_identifiers.add(check_val)
        
            ### [CREATION+ASSIGN] station group does not exist
            # obtain station group ID
            station_group_id = self._obtain_station_group_id()
            # create new station group
            station_group = StationGroup(
                                        env=self, group_id=station_group_id, 
                                        custom_identifier=custom_group_id, 
                                        name=station_group_name)
            # add proc station to associated ones
            station_group.add_processing_station(processing_station=processing_station)
            
            # new entry for station group database
            new_entry: DataFrame = pd.DataFrame({
                                    'station_group_id': [station_group_id],
                                    'custom_group_id': [custom_group_id],
                                    'station_group_name': [station_group_name],
                                    'station_group': [station_group]})
            new_entry = new_entry.astype(self._station_group_prop)
            new_entry = new_entry.set_index('station_group_id')
            self._station_group_db = pd.concat([self._station_group_db, new_entry])
            
        logger_env.info(f"Successfully registered processing station {processing_station} \
            with station group {station_group}")
            
        return station_group_id, station_group
    
    def get_station_group_by_prop(
        self,
        val: CustomID | str,
        property: str = 'station_group_id',
        target_prop: str = 'station_group',
    ) -> StationGroup:
        """
        obtain a station group by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._station_group_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._station_group_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        if property == 'station_group_id':
            # direct indexing for ID property; env_id always unique, no need for duplicate check
            try:
                temp1: StationGroup = self._station_group_db.loc[val, target_prop]
                return temp1
            except KeyError:
                raise IndexError(f"There were no resources found for the property '{property}' \
                                with the value '{val}'")
        else:
            temp1: Series = self._station_group_db.loc[self._station_group_db[property] == val, target_prop]
            # check for empty search result, at least one result necessary
            if len(temp1) == 0:
                raise IndexError(f"There were no resources found for the property '{property}' \
                                with the value '{val}'")
            # check for multiple entries with same prop-value pair
            ########### PERHAPS CHANGE NECESSARY
            ### multiple entries but only one returned --> prone to errors
            elif len(temp1) > 1:
                # warn user
                logger_env.warning(f"CAUTION: There are multiple resources which share the \
                            same value '{val}' for the property '{property}'. \
                            Only the first entry is returned.")
        
            return temp1.iat[0]
    
    @property
    def sinks(self) -> set[Sink]:
        """registered sinks"""
        return self._sinks
    
    def update_res_state(
        self,
        obj: InfrastructureObject,
        state: str,
        reset_temp: bool = False,
    ) -> None:
        """method to update the state of a resource object in the resource database"""
        # update resource database
        logger_env.debug(f"Set state of {obj} to {state}")
        # update state tracking of the job instance
        logger_env.debug(f"[Object:{self}]: Monitor is {obj.stat_monitor}")
        
        # check if 'TEMP' state should be reset
        if reset_temp:
            # special reset method, calls state setting to previous state
            obj.stat_monitor.reset_temp_state()
            state = obj.stat_monitor.state_current
        else:
            obj.stat_monitor.set_state(state=state)
        
        self._res_db.loc[obj.env_id, 'state'] = state
        logger_env.debug(f"Executed state setting of {obj} to {state}")
        
        return None
    
    @property
    def res_db(self) -> DataFrame:
        """obtain a current overview of registered objects in the environment"""
        return self._res_db
    
    @property
    def station_group_db(self) -> DataFrame:
        return self._station_group_db

    #@lru_cache(maxsize=200)
    def get_res_obj_by_prop(
        self,
        val: EnvID | CustomID | str,
        property: str = 'env_id',
        target_prop: str = 'resource',
    ) -> InfrastructureObject:
        """
        obtain a resource object from the environment by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._res_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._res_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        if property == 'env_id':
            # direct indexing for ID property; env_id always unique, no need for duplicate check
            try:
                temp1: InfrastructureObject = self._res_db.loc[val, target_prop]
                return temp1
            except KeyError:
                raise IndexError(f"There were no resources found for the property '{property}' \
                                with the value '{val}'")
        else:
            temp1: Series = self._res_db.loc[self._res_db[property] == val, target_prop]
            # check for empty search result, at least one result necessary
            if len(temp1) == 0:
                raise IndexError(f"There were no resources found for the property '{property}' \
                                with the value '{val}'")
            # check for multiple entries with same prop-value pair
            ########### PERHAPS CHANGE NECESSARY
            ### multiple entries but only one returned --> prone to errors
            elif len(temp1) > 1:
                # warn user
                logger_env.warning(f"CAUTION: There are multiple resources which share the \
                            same value '{val}' for the property '{property}'. \
                            Only the first entry is returned.")
        
            return temp1.iat[0]
        
    def check_integrity(self) -> None:
        """
        method to evaluate if certain criteria for the simulation run are satisfied
        checks for:
        - registered dispatcher (min: 1, max: 1)
        - registered sink (min: 1, max: INF)
        """
        if not self._dispatcher_registered:
            raise ValueError("No Dispatcher instance registered.")
        elif not self._sink_registered:
            raise ValueError("No Sink instance registered.")
        
        logger_env.info(f"Integrity check for Environment {self.name()} successful.")
        
        return None
        
    def res_objs_temp_state(
        self,
        res_objs: Iterable[InfrastructureObject],
        reset_temp: bool,
    ) -> None:
        """Sets/resets given resource objects from the 'TEMP' state

        Parameters
        ----------
        res_objs : tuple[InfrastructureObject]
            objects for which the TEMP state should be changed
        set_temp : bool
            indicates if the temp state should be set or reset
        """
        for obj in res_objs:
            self.update_res_state(obj=obj, state='TEMP', reset_temp=reset_temp)
            # calculate KPIs if 'TEMP' state is set
            if not reset_temp:
                obj.stat_monitor.calc_KPI()
        
        return None
    
    def finalise_sim(self) -> None:
        """
        Function which should be executed at the end of the simulation.
        Can be used for finalising data collection, other related tasks or further processing pipelines
        """
        # set end state for each resource object to calculate the right time amounts
        for res_obj in self._res_db['resource']:
            res_obj.finalise()
        logger_env.info("Finalisation of the state information for all resource objects successful.")
        
        # dispatcher instance
        self._dispatcher.finalise()


[Jump to top](#top)

---
### **Infrastructure Objects**

*Adapting machine class:*
- user defined process function
- env related stuff in generic class

<a id='infrastructureobject'></a>

In [721]:
class Monitor(object):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        obj: InfrastructureObject | Job | Operation,
        init_state: str = 'INIT',
        possible_states: Iterable[str] = (
            'INIT',
            'FINISH',
            'TEMP',
            'WAITING', 
            'PROCESSING', 
            'BLOCKED', 
            'FAILED', 
            'PAUSED',
        ),
        **kwargs,
    ) -> None:
        """
        Class to monitor associated objects (load and resource)
        """
        # initialise parent class if available
        super().__init__(**kwargs)
        
        # [REGISTRATION]
        self.env = env
        self._target_object = obj
        
        # [STATE] state parameters
        # all possible/allowed states
        self.states_possible: set[str] = set(possible_states)
        # always add states 'INIT', 'FINISH', 'TEMP' for control flow
        if not 'INIT' in self.states_possible:
            self.states_possible.add('INIT')
        if not 'FINISH' in self.states_possible:
            self.states_possible.add('FINISH')
        if not 'TEMP' in self.states_possible:
            self.states_possible.add('TEMP')
            
        # check integrity of the given state
        if init_state in self.states_possible:
            self.state_current: str = init_state
        else:
            raise ValueError(f"The state {state} is not allowed. Must be one of {self.states_possible}")
        
        # boolean indicator if a state is set
        self.state_status: dict[str, bool] = dict()
        # time counter for each state
        self.state_times: dict[str, float] = dict()
        # starting time variable indicating when the last state assignment took place
        self.state_starting_time: float = env.now()
        
        for state in self.states_possible:
            # init state time dictionary
            self.state_times[state] = 0.
            # init state is set to True
            if state == self.state_current:
                self.state_status[state] = True
            else:
                self.state_status[state] = False
                
        # DataFrame to further analyse state durations
        self.state_durations: DataFrame | None = None

        # availability indicator
        self._availability_states: set[str] = set([
            'WAITING',
        ])
        if self.state_current in self._availability_states:
            self.is_available: bool = True
        else:
            self.is_available: bool = False
        
        # additional 'TEMP' state information
        # indicator if state was 'TEMP'
        self._is_temp: bool = False
        # state before 'TEMP' was set
        self._state_before_temp: str = self.state_current
        # time components
        self.time_active: float = 0.
        """
        self.time_occupied: float = 0.
        
        # resource KPIs
        self.utilisation: float = 0.
        
        # logistic objective values
        self.WIP_load_time: float = 0.
        self.WIP_load_num_jobs: int = 0
        
        """
        
        return None
        
    def get_current_state(self) -> str:
        """get the current state of the associated resource"""
        return self.state_current
        
    def set_state(
        self,
        state: str,
    ) -> None:
        """
        function to set the object in the given state
        state: name of the state in which the object should be placed, must be part \
            of the object's possible states
        """
        # eliminate lower-case letter
        target_state = state.upper()
        
        # check if state is allowed
        if target_state not in self.states_possible:
            raise ValueError(f"The state {target_state} is not allowed. Must be one of {self.states_possible}")
        
        # check if state is already set
        if self.state_status[target_state] == True and target_state != 'TEMP':
            logger_monitors.info(f"Tried to set state of {self._target_object} to >>{target_state}<<, but this state was already set.\
                The object's state was not changed.")
        # check if the 'TEMP' state was already set, this should never happen
        # if it happens raise an error to catch wrong behaviour
        elif self.state_status[target_state] == True and target_state != 'TEMP':
            raise RuntimeError(f"Tried to set state of {self._target_object} to >>TEMP<<, but this state was already set.")
        
        # calculate time for which the object was in the current state before changing it
        current_state_start = self.state_starting_time
        current_time = self.env.now()
        current_state_duration = current_time - current_state_start
        # add time to the time counter for the current state
        current_state = self.state_current
        self.state_times[current_state] += current_state_duration
        
        # check if 'TEMP' state shall be set
        if target_state == 'TEMP':
            # set 'TEMP' state indicator to true
            self._is_temp = True
            # save current state for the state reset
            self._state_before_temp = current_state
        
        # set old state to False and new state to True
        self.state_status[current_state] = False
        self.state_status[target_state] = True
        # assign new state as current one
        self.state_current = target_state
        # set state starting time to current time
        self.state_starting_time = current_time
        # availability
        if self.state_current in self._availability_states:
            self.is_available: bool = True
        elif self.state_current == 'TEMP':
            # 'TEMP' state shall not change the availability indicator
            pass
        else:
            self.is_available: bool = False
        
        logger_monitors.debug(f"Duration for state {current_state} on {self._target_object} was {current_state_duration}")
        
        return None
    
    def reset_temp_state(self) -> None:
        """Reset from 'TEMP' state
        """
        # check if object was in TEMP state, raise error if not
        if not self._is_temp:
            raise RuntimeError(f"Tried to reset {self._target_object} from 'TEMP' state but \
                the current state is >>{self.state_current}<<")
        else:
            self._is_temp = False
            self.set_state(state=self._state_before_temp)
            
        return None
    
    def calc_KPI(
        self,
        is_finalise: bool = False,
    ) -> None:
        """calculates different KPIs at any point in time
        """
        
        # state durations for analysis
        if not is_finalise:
            self.state_durations = self.state_durations_as_df()
        
        # [TOTAL ACTIVATE TIME]
        self.time_active = self.state_durations.loc[:, 'abs [timesteps]'].sum()
        
        """
        # PROCESSING STATIONS ONLY
        if isinstance(self._target_object, ProcessingStation):
            # [OCCUPATION]
            # properties which count as occupied
            # paused counts in because pausing the processing station is an external factor
            util_props = ['PROCESSING', 'PAUSED']
            self.time_occupied = state_durations.loc[util_props, 'abs [timesteps]'].sum()
            
            # [UTILISATION]
            # avoid division by 0
            if self.time_active > 0.:
                self.utilisation = self.time_occupied / self.time_active
        """
        
        return None
    
    """
    def change_WIP(
        self,
        job: Job,
        remove: bool,
    ) -> None:
        
        # removing WIP
        if remove:
            # next operation of the job already assigned
            self.WIP_load_time -= job.last_proc_time
            self.WIP_load_num_jobs -= 1
        else:
            self.WIP_load_time += job.current_proc_time
            self.WIP_load_num_jobs += 1
        
        return None
    """
    
    def state_durations_as_df(self) -> DataFrame:
        """Calculates absolute and relative state durations at the current time

        Returns
        -------
        DataFrame
            State duration table with absolute and relative values
        """
        # build state duration table
        temp1: Series = pd.Series(data=self.state_times)
        temp2: DataFrame = temp1.to_frame()
        temp2.columns = ['abs [timesteps]']
        temp2['rel [%]'] = temp2['abs [timesteps]'] / temp2.sum(axis=0)['abs [timesteps]'] * 100
        temp2 = temp2.drop(labels=['INIT', 'FINISH', 'TEMP'], axis=0)
        temp2 = temp2.sort_index(axis=0, ascending=True, kind='stable')
        state_durations_df = temp2.copy()
        
        return state_durations_df
    
    def finalise_stats(self) -> None:
        """finalisation of stats gathering"""
        
        # assign state duration table
        self.state_durations = self.state_durations_as_df()
        
        # calculate KPIs
        self.calc_KPI(is_finalise=True)
        
        return None
    
    ### ANALYSE AND CHARTS ###
    def draw_state_bar_chart(        
        self,
        save_img: bool = False,
        save_html: bool = False,
        file_name: str = 'state_distribution',
    ) -> PlotlyFigure:
        """draws the collected state times of the object as bar chart"""
        data = pd.DataFrame.from_dict(data=self.state_times, orient='index', columns=['total time'])
        data.index = data.index.rename('state')
        data = data.sort_index(axis=0, kind='stable')
        
        fig: PlotlyFigure = px.bar(data, text_auto='.2f')
        fig.update_layout(title=f'State Time Distribution of {self._target_object}', showlegend=False)
        fig.update_yaxes(title=dict({'text': 'total time'}))
        
        fig.show()
        
        file_name = file_name + f'_{self}'
        
        if save_html:
            file = f'{file_name}.html'
            fig.write_html(file)
        
        if save_img:
            file = f'{file_name}.svg'
            fig.write_image(file)
        
        return fig

In [722]:
class BufferMonitor(Monitor):
    
    def __init__(
        self,
        obj: Buffer,
        **kwargs,
    ) -> None:
        # initialise parent class
        super().__init__(obj=obj, **kwargs)
        
        # fill level tracking
        self._level_db_types = {
            'sim_time': float,
            'duration': float,
            'level': int,
        }
        self._level_db: DataFrame = pd.DataFrame(
                                        columns=['sim_time', 'duration', 'level'], 
                                        data=[[0., 0., obj.start_fill_level]])
        self._level_db = self._level_db.astype(self._level_db_types)
        
        self._current_fill_level = obj.start_fill_level
        self._fill_level_starting_time: float = self.env.now()
        self._wei_avg_fill_level: float | None = None
        
    @property
    def wei_avg_fill_level(self) -> float:
        return self._wei_avg_fill_level
    
    @property
    def level_db(self) -> DataFrame:
        return self._level_db
    
    def set_state(
        self,
        state: str,
    ) -> None:
        """additional level tracking functionality"""
        super().set_state(state=state)
        
        is_finalise: bool = False
        if self.state_current == 'FINISH':
            is_finalise: bool = True
        self.track_fill_level(is_finalise=is_finalise)
        
    # Buffer fill level tracking
    def track_fill_level(
        self,
        is_finalise: bool = False,
    ) -> None:
        """adds an entry to the fill level database"""
        # only calculate duration if buffer level changes
        current_time = self.env.now()
        duration = current_time - self._fill_level_starting_time
        logger_buffers.debug(f"[BUFFER: {self._target_object}] Current time is {current_time} with level {len(self._target_object)} and old level {self._current_fill_level}")
        #if ((self._current_fill_level != len(self)) and (duration > 0.0)) or is_finalise:
        if (self._current_fill_level != len(self._target_object)) or is_finalise:
            temp1: Series = pd.Series(
                                    index=['sim_time', 'duration', 'level'],
                                    data=[current_time, duration, self._current_fill_level])
            temp2: DataFrame = temp1.to_frame().T.astype(self._level_db_types)
            self._level_db = pd.concat([self._level_db, temp2], ignore_index=True)
            self._current_fill_level = len(self._target_object)
            self._fill_level_starting_time = current_time
        
        return None
        
    def finalise_stats(self) -> None:
        """finalisation of stats gathering"""
        # execute parent class function
        super().finalise_stats()
        
        # finalise fill level tracking
        self.track_fill_level(is_finalise=True)
        
        # weighted average fill level
        self._level_db = self._level_db.loc[self._level_db['duration'] > 0., :].copy()
        self._level_db = self._level_db.reset_index(drop=True)
        temp1: DataFrame = self._level_db.copy()
        temp1['mul'] = temp1['duration'] * temp1['level']
        sums: Series = temp1.sum(axis=0)
        self._wei_avg_fill_level: float = sums['mul'] / sums['duration']
        
    ### ANALYSE AND CHARTS ###
    def draw_fill_level(
        self,
        save_img: bool = False,
        save_html: bool = False,
        file_name: str = 'fill_level',
    ) -> PlotlyFigure:
        """
        method to draw and display the fill level expansion of the corresponding buffer
        """
        # add starting point to start chart at t = init time
        data = self.level_db.copy()
        val1: float = data.loc[0, 'sim_time'] - data.loc[0, 'duration']
        val2: float = 0.
        val3: int = data.loc[0, 'level']
        temp1: DataFrame = pd.DataFrame(columns=data.columns, data=[[val1, val2, val3]])
        temp1 = pd.concat([temp1, data], ignore_index=True)
        
        fig: PlotlyFigure = px.line(x=temp1['sim_time'], y=temp1['level'], line_shape="vh")
        fig.update_traces(line=dict(width=3))
        fig.update_layout(title=f'Fill Level of {self._target_object}')
        fig.update_yaxes(title=dict({'text': 'fill level [-]'}))
        fig.update_xaxes(title=dict({'text': 'time'}))
        # weighted average fill level
        fig.add_hline(
                    y=self.wei_avg_fill_level, line_width=3, 
                    line_dash='dot', line_color='orange')
        # capacity
        cap = self._target_object.capacity()
        if cap < INF:
            fig.add_hline(
                        y=cap, line_width=3, 
                        line_dash='dash', line_color='red')
        
        fig.show()
        
        file_name = file_name + f'_{self}'
        
        if save_html:
            file = f'{file_name}.html'
            fig.write_html(file)
        
        if save_img:
            file = f'{file_name}.svg'
            fig.write_image(file)
        
        return fig

In [723]:
class ProcStationMonitor(Monitor):
    
    def __init__(
        self,
        obj: ProcessingStation,
        **kwargs,
    ) -> None:
        # initialise parent class
        super().__init__(obj=obj, **kwargs)
        
        # WIP tracking time load
        self._WIP_time_db_types = {
            'sim_time': float,
            'duration': float,
            'level': float,
        }
        ###################### PERHAPS ADD STARTING LEVEL LATER
        self._WIP_time_db: DataFrame = pd.DataFrame(
                                        columns=['sim_time', 'duration', 'level'], 
                                        data=[[0., 0., 0.]])
        self._WIP_time_db = self._WIP_time_db.astype(self._WIP_time_db_types)
        
        # WIP tracking number of jobs
        self._WIP_num_db_types = {
            'sim_time': float,
            'duration': float,
            'level': int,
        }
        ###################### PERHAPS ADD STARTING LEVEL LATER
        self._WIP_num_db: DataFrame = pd.DataFrame(
                                        columns=['sim_time', 'duration', 'level'], 
                                        data=[[0., 0., 0]])
        self._WIP_num_db = self._WIP_num_db.astype(self._WIP_num_db_types)
        
        #self._current_WIP_time: float = 0.
        #self._last_WIP_time: float = 0.
        #self._current_WIP_num: int = 0
        #self._last_WIP_num: int = 0
        
        self._WIP_time_starting_time: float = self.env.now()
        self._WIP_num_starting_time: float = self.env.now()
        self._wei_avg_WIP_level_time: float | None = None
        self._wei_avg_WIP_level_num: float | None = None
        
        # time components
        self.time_occupied: float = 0.
        
        # resource KPIs
        self.utilisation: float = 0.
        
        # logistic objective values
        self.WIP_load_time: float = 0.
        self._WIP_load_time_last: float = 0.
        self.WIP_load_num_jobs: int = 0
        self._WIP_load_num_jobs_last: int = 0
        
        return None
    
    @property
    def wei_avg_WIP_level_time(self) -> float:
        return self._wei_avg_WIP_level_time
    
    @property
    def wei_avg_WIP_level_num(self) -> float:
        return self._wei_avg_WIP_level_num
    
    @property
    def WIP_time_db(self) -> DataFrame:
        return self._WIP_time_db
    
    @property
    def WIP_num_db(self) -> DataFrame:
        return self.__WIP_num_db
    
    def track_WIP_level(
        self,
        is_finalise: bool = False,
    ) -> None:
        """adds an entry to the fill level database"""
        # only calculate duration if buffer level changes
        current_time = self.env.now()
        
        if (self._WIP_load_time_last != self.WIP_load_time) or is_finalise:
            duration = current_time - self._WIP_time_starting_time
            temp1: Series = pd.Series(
                                    index=['sim_time', 'duration', 'level'],
                                    data=[current_time, duration, self.WIP_load_time])
            temp2: DataFrame = temp1.to_frame().T.astype(self._WIP_time_db_types)
            self._WIP_time_db = pd.concat([self._WIP_time_db, temp2], ignore_index=True)
            self._WIP_load_time_last = self.WIP_load_time
            self._WIP_time_starting_time = current_time
            
        if (self._WIP_load_num_jobs_last != self.WIP_load_num_jobs) or is_finalise:
            duration = current_time - self._WIP_num_starting_time
            temp1: Series = pd.Series(
                                    index=['sim_time', 'duration', 'level'],
                                    data=[current_time, duration, self.WIP_load_num_jobs])
            temp2: DataFrame = temp1.to_frame().T.astype(self._WIP_num_db_types)
            self._WIP_num_db = pd.concat([self._WIP_num_db, temp2], ignore_index=True)
            self._WIP_load_num_jobs_last = self.WIP_load_num_jobs
            self._WIP_num_starting_time = current_time
        
        return None
    
    def calc_KPI(
        self,
        is_finalise: bool = False,
    ) -> None:
        
        super().calc_KPI()
        
        # [OCCUPATION]
        # properties which count as occupied
        # paused counts in because pausing the processing station is an external factor
        util_props = ['PROCESSING', 'PAUSED']
        self.time_occupied = self.state_durations.loc[util_props, 'abs [timesteps]'].sum()
        
        # [UTILISATION]
        # avoid division by 0
        if self.time_active > 0.:
            self.utilisation = self.time_occupied / self.time_active
        
        return None
    
    def change_WIP(
        self,
        job: Job,
        remove: bool,
    ) -> None:
        
        # removing WIP
        if remove:
            # next operation of the job already assigned
            self.WIP_load_time -= job.last_proc_time
            self.WIP_load_num_jobs -= 1
        else:
            self.WIP_load_time += job.current_proc_time
            self.WIP_load_num_jobs += 1
            
        self.track_WIP_level()
        
        return None
        
    def finalise_stats(self) -> None:
        """finalisation of stats gathering"""
        # execute parent class function
        super().finalise_stats()
        
        # finalise WIP level tracking
        self.track_WIP_level(is_finalise=True)
        
        # weighted average WIP time level
        self._WIP_time_db = self._WIP_time_db.loc[self._WIP_time_db['duration'] > 0., :].copy()
        self._WIP_time_db = self._WIP_time_db.reset_index(drop=True)
        temp1: DataFrame = self._WIP_time_db.copy()
        temp1['mul'] = temp1['duration'] * temp1['level']
        sums: Series = temp1.sum(axis=0)
        self._wei_avg_WIP_level_time: float = sums['mul'] / sums['duration']
        # weighted average WIP num level
        self._WIP_num_db = self._WIP_num_db.loc[self._WIP_num_db['duration'] > 0., :].copy()
        self._WIP_num_db = self._WIP_num_db.reset_index(drop=True)
        temp1: DataFrame = self._WIP_num_db.copy()
        temp1['mul'] = temp1['duration'] * temp1['level']
        sums: Series = temp1.sum(axis=0)
        self._wei_avg_WIP_level_num: float = sums['mul'] / sums['duration']
        
    ### ANALYSE AND CHARTS ###
    def draw_WIP_level(
        self,
        use_num_jobs_metric: bool = False,
        save_img: bool = False,
        save_html: bool = False,
        file_name: str = 'fill_level',
    ) -> PlotlyFigure:
        """
        method to draw and display the fill level expansion of the corresponding buffer
        """
        # add starting point to start chart at t = init time
        if use_num_jobs_metric:
            data = self._WIP_num_db.copy()
            title = f'WIP Level Num Jobs of {self._target_object}'
            yaxis = 'WIP Level Number of Jobs [-]'
            avg_WIP_level = self._wei_avg_WIP_level_num
        else:
            data = self._WIP_time_db.copy()
            title = f'WIP Level Time of {self._target_object}'
            yaxis = 'WIP Level Time [time units]'
            avg_WIP_level = self._wei_avg_WIP_level_time
        val1: float = data.loc[0, 'sim_time'] - data.loc[0, 'duration']
        val2: float = 0.
        val3: int = data.loc[0, 'level']
        temp1: DataFrame = pd.DataFrame(columns=data.columns, data=[[val1, val2, val3]])
        temp1 = pd.concat([temp1, data], ignore_index=True)
        
        fig: PlotlyFigure = px.line(x=temp1['sim_time'], y=temp1['level'], line_shape="vh")
        fig.update_traces(line=dict(width=3))
        fig.update_layout(title=title)
        fig.update_yaxes(title=dict({'text': yaxis}))
        fig.update_xaxes(title=dict({'text': 'time'}))
        # weighted average WIP level
        fig.add_hline(
                    y=avg_WIP_level, line_width=3, 
                    line_dash='dot', line_color='orange')
        
        fig.show()
        
        file_name = file_name + f'_{self}'
        
        if save_html:
            file = f'{file_name}.html'
            fig.write_html(file)
        
        if save_img:
            file = f'{file_name}.svg'
            fig.write_image(file)
        
        return fig

**Implement an ABC to provide an interface for the simulation logic methods:**
- *name: ResourceModule*
- pre, main, post
- finalise

In [724]:
class StationGroup(set):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        group_id: ObjectID,
        custom_identifier: CustomID,
        name: str | None = None,
        **kwargs,
    ) -> None:
        """Group of processing stations which are considered parallel machines

        Parameters
        ----------
        env : SimulationEnvironment
            _description_
        custom_identifier : CustomID
            _description_
        name : str | None, optional
            _description_, by default None
        """
        # initiliase base class
        super().__init__(**kwargs)
        
        # assign basic information
        self._env = env
        self._group_id = group_id
        self._custom_identifier = custom_identifier
        self._name = name
        
    def __repr__(self) -> str:
        return f'StationGroup(custom_id: {self._custom_identifier}, name: {self._name})'
    
    @property
    def env(self) -> SimulationEnvironment:
        return self._env
    
    @property
    def group_id(self) -> ObjectID:
        return self._group_id
    
    @property
    def custom_identifier(self) -> CustomID:
        return self._custom_identifier
    
    @property
    def name(self) -> str | None:
        return self._name
    
    def as_list(self) -> list[ProcessingStation]:
        """output the associated processing stations as list

        Returns
        -------
        list[ProcessingStation]
            list of associated processing stations
        """
        return list(self)
    
    def as_tuple(self) -> tuple[ProcessingStation]:
        """output the associated processing stations as tuple

        Returns
        -------
        tuple[ProcessingStation]
            tuple of associated processing stations
        """
        return tuple(self)
    
    def add_processing_station(
        self,
        processing_station: ProcessingStation,
    ) -> None:
        """adds a processing station to itself

        Parameters
        ----------
        processing_station : ProcessingStation
            _description_
        """
        if not processing_station in self:
            self.add(processing_station)
        else:
            raise UserWarning(f"Processing station {processing_station} was \
                already in station group {self}!")

In [725]:
class InfrastructureObject(sim.Component):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        custom_identifier: CustomID,
        name: str | None = None,
        capacity: float = INF,
        state: str = 'INIT',
        possible_states: Iterable[str] = (
            'INIT',
            'FINISH',
            'TEMP',
            'WAITING', 
            'PROCESSING', 
            'BLOCKED', 
            'FAILED', 
            'PAUSED',
        ),
        **kwargs,
    ) -> None:
        """
        env: simulation environment in which the infrastructure object is embedded
        custom_identifier: unique user-defined custom ID of the given object \
            necessary for user interfaces
        capacity: capacity of the infrastructure object, if multiple processing \
            slots available at the same time > 1, default=1
        """
        # [STATS] Monitoring
        # special buffer monitor
        if isinstance(self, Buffer):
            self._stat_monitor = BufferMonitor(
                                env=env, obj=self, init_state=state, 
                                possible_states=possible_states, **kwargs)
        elif isinstance(self, ProcessingStation):
            self._stat_monitor = ProcStationMonitor(
                                env=env, obj=self, init_state=state, 
                                possible_states=possible_states, **kwargs)
        else:
            self._stat_monitor = Monitor(env=env, obj=self, init_state=state, 
                                possible_states=possible_states, **kwargs)
        
        # assert machine information and register object in the environment
        current_state = self._stat_monitor.get_current_state()
        self._env_id, name = env.register_resource(
                                obj=self, custom_identifier=custom_identifier,
                                name=name, state=current_state)
        self.custom_identifier = custom_identifier
        self.cap = capacity
        # intialize base class
        process = 'main_logic'
        super().__init__(env=env, name=name, process=process, **kwargs)
        
        # add logic queues
        # each resource uses one associated logic queue, logic queues are not physically available
        queue_name: str = f"queue_{self.name()}"
        self.logic_queue: Queue = sim.Queue(name=queue_name, env=self.env)
        
        # currently available jobs on that resource
        self.contents: OrderedDict[ObjectID, Job] = OrderedDict()
        
        # [STATS] additional information
        # number of inputs/outputs
        self.num_inputs: int = 0
        self.num_outputs: int = 0
    
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    
    @property
    def stat_monitor(self) -> Monitor:
        return self._stat_monitor
    
    def add_content(
        self,
        job: Job,
    ) -> None:
        """add contents to the InfrastructureObject"""
        job_id = job.job_id
        if job_id not in self.contents:
            self.contents[job_id] = job
        else:
            raise KeyError(f"Job {job} already in contents of {self}")
    
    def remove_content(
        self,
        job: Job,
    ) -> None:
        """remove contents from the InfrastructureObject"""
        job_id = job.job_id
        if job_id in self.contents:
            del self.contents[job_id]
        else:
            raise KeyError(f"Job {job} not in contents of {self}")
    
    def put_job(
        self,
        job: Job,
    ) -> InfrastructureObject | None:
        """
        placing
        """
        # ALLOCATION REQUEST
        ## call dispatcher --> request for allocation
        ## self._dispatcher.request_allocation ...
        ### input job
        #### LATER: LOGIC FOR RESOURCE ALLOCATION (AGENT)
        ### - Dispatcher calls "get_next_operation"
        ### - Dispatcher returns target_machine
        ## ret: obtaining target machine
        # ++++++++++ add later ++++++++++++
        ## time component: given start date of operation
        ## returning release date, waiting for release date or release early
        dispatcher = self.env.dispatcher
        target_station = dispatcher.request_job_allocation(job=job)
        # get logic queue
        logic_queue = target_station.logic_queue
        # check if the target is a sink
        if isinstance(target_station, Sink):
            pass
        else:
            # check if associated buffers exist
            logger_prodStations.debug(f"[{self}] Check for buffers")
            buffers = target_station.buffers
            
            if buffers:
                #logger_prodStations.debug(f"[{self}] Buffer found")
                # [STATE:InfrStructObj] BLOCKED
                self.env.update_res_state(obj=self, state='BLOCKED')
                # [STATE:Job] BLOCKED
                dispatcher.update_job_state(job=job, state='BLOCKED')
                yield self.to_store(store=buffers, item=job, fail_delay=FAIL_DELAY, fail_priority=1)
                if self.failed():
                    raise UserWarning(f"Store placement failed after {FAIL_DELAY} time steps. \
                        There seems to be deadlock.")
                # [STATE:Buffer] trigger state setting for target buffer
                buffer = self.to_store_store()
                if not isinstance(buffer, Buffer):
                    #logger_prodStations.debug(f"To store store object: {buffer}")
                    raise TypeError(f"From {self}: Job {job} Obj {buffer} is no buffer type at {self.env.now()}")
                buffer.activate()
                # [CONTENT:Buffer] add content
                buffer.add_content(job=job)
                # [STATS:Buffer] count number of inputs
                buffer.num_inputs += 1
                logger_prodStations.debug(f"obj = {self} \t type of buffer >>{buffer}<< = {type(buffer)} at {self.env.now()}")
            else:
                # adding request to machine
                # currently not possible because machines are components,
                # but resources which could be requested are not
                pass
        
        # [Job] enter logic queue after physical placement
        job.enter(logic_queue)
        # [STATS:WIP] REMOVING WIP FROM CURRENT STATION
        # remove only if it was added before, only case if the last operation exists
        if job.last_op is not None:
            self.stat_monitor.change_WIP(job=job, remove=True)
        # [STATS:WIP] ADDING WIP TO TARGET STATION
        # add only if there is a next operation, only case if the current operation exists
        if job.current_op is not None:
            target_station.stat_monitor.change_WIP(job=job, remove=False)
        
        # activate target processing station if passive
        if target_station.ispassive():
            target_station.activate()
        
        logger_prodStations.debug(f"[{self}] Put Job {job} in queue {logic_queue}")
    
        # [STATE:InfrStructObj] WAITING
        #print(f"-------> {target_station=}, {job=}, {type(target_station)=}")
        self.env.update_res_state(obj=self, state='WAITING')
        # [STATE:Job] successfully placed --> WAITING
        dispatcher.update_job_state(job=job, state='WAITING')
        # [STATS:InfrStructObj] count number of ouputs
        self.num_outputs += 1
        
        return target_station
    
    def get_job(self) -> tuple[Job, float]:
        """
        getting jobs from associated predecessor resources
        """
        # entering target machine (logic_buffer)
        ## logic_buffer: job queue regardless of physical buffers
        ### entity physically on machine, but no true holding resource object (violates load-resource model)
        ### no capacity restrictions between resources, e.g. source can endlessly produce entities
        ## --- logic ---
        ## job enters logic queue of machine with unrestricted capacity
        ## each machine can have an associated physical buffer
        dispatcher = self.env.dispatcher
        # request job from associated queue
        job, job_proc_time = dispatcher.request_job_sequencing(req_obj=self)
        
        # request and get job from associated buffer if it exists
        if self._buffers:
            yield self.from_store(store=self._buffers, filter=lambda item: item.job_id == job.job_id)
            buffer = self.from_store_store()
            # [STATS:Buffer] count number of outputs
            buffer.num_outputs += 1
            # [CONTENT:Buffer] remove content
            buffer.remove_content(job=job)
            # [STATE:Buffer] trigger state setting for target buffer
            buffer.activate()
        else:
            pass
        
        # [STATE:InfrStructObj] set state to processing
        self.env.update_res_state(obj=self, state='PROCESSING')
        # [STATE:Job] successfully taken --> PROCESSING
        dispatcher.update_job_state(job=job, state='PROCESSING')
        
        # [STATS:InfrStructObj] count number of inputs
        self.num_outputs += 1
        
        return job, job_proc_time
    
    ### PROCESS LOGIC
    # each method of 'pre_process', 'sim_control', 'post_process' must be implemented in the child classes
    def pre_process(self) -> None:
        """return type: tuple with parameters or None"""
        raise NotImplementedError(f"No pre-process method for {self} of type {self.__class__.__name__} defined.")
    
    def sim_control(self) -> None:
        """return type: tuple with parameters or None"""
        raise NotImplementedError(f"No sim-control method for {self} of type {self.__class__.__name__} defined.")
    
    def post_process(self) -> None:
        """return type: tuple with parameters or None"""
        raise NotImplementedError(f"No post-process method for {self} of type {self.__class__.__name__} defined.")
    
    def main_logic(self) -> Iterator[Any]:
        """main logic loop for all resources in the simulation environment"""
        logger.debug(f"----> Process logic of {self}")
        # pre control logic
        ret = self.pre_process()
        # main control logic
        if ret is not None:
            ret = yield from self.sim_control(*ret)
        else:
            ret = yield from self.sim_control()
        # post control logic
        if ret is not None:
            ret = self.post_process(*ret)
        else:
            ret = self.post_process()
            
    def finalise(self) -> None:
        """
        method to be called at the end of the simulation run by 
        the environment's "finalise_sim" method
        """
        # set finish state for each infrastructure object no matter of which child class
        self.env.update_res_state(obj=self, state='FINISH')
        # finalise stat gathering
        self._stat_monitor.finalise_stats()
    

<a id='processingstation'></a>

In [726]:
class ProcessingStation(InfrastructureObject):
    
    def __init__(
        self,
        buffers: Iterable[Buffer] | None = None,
        station_group_custom_id: CustomID | None = None,
        station_group_name: str | None = None,
        **kwargs,
    ) -> None:
        """
        env: simulation environment in which the infrastructure object is embedded
        capacity: capacity of the infrastructure object, if multiple processing \
            slots available at the same time > 1, default=1
        """
        # intialize base class
        super().__init__(**kwargs)
        
        # station groups
        (self._station_group_id, 
         self._station_group) = self.env.register_station_group(
                                                        processing_station=self,
                                                        custom_group_id=station_group_custom_id,
                                                        station_group_name=station_group_name)
        
        # add physical buffers, more than one allowed
        # contrary to logic queues buffers are infrastructure objects and exist physically
        if buffers is None:
            self._buffers: set[Buffer] = set()
        else:
            self._buffers: set[Buffer] = set(buffers).copy()
        
        # add processing station to the associated ones of each buffer
        # necessary because if the number of resources for one buffer exceeds its capacity
        # deadlocks are possible
        for buffer in self._buffers:
            buffer.add_prod_station(prod_station=self)
            
        return None
    
    @property
    def station_group_id(self) -> ObjectID:
        return self._station_group_id
    
    @property
    def station_group(self) -> StationGroup:
        return self._station_group
    
    @property
    def buffers(self) -> set[Buffer]:
        return self._buffers
    
    def add_buffer(
        self,
        buffer: Buffer,
    ) -> None:
        """
        adding buffer to the current associated ones
        """
        # only buffer types allowed
        if not isinstance(buffer, Buffer):
            raise TypeError(f"Object is no Buffer type. Only objects of type Buffer can be added as buffers.")
        # check if already present
        if buffer not in self._buffers:
            self._buffers.add(buffer)
            buffer.add_prod_station(prod_station=self)
        else:
            logger_prodStations.warning(f"The Buffer >>{buffer}<< is already associated with the resource >>{self}<<. \
                Buffer was not added to the resource.")

    def remove_buffer(
        self,
        buffer: Buffer,
    ) -> None:
        """
        removing buffer from the current associated ones
        """
        if buffer in self._buffers:
            self._buffers.remove(buffer)
            buffer.remove_prod_station(prod_station=self)
        else:
            raise KeyError(f"The buffer >>{buffer}<< is not associated with the resource >>{self}<< and \
                therefore could not be removed.")
    
    ### PROCESS LOGIC
    def pre_process(self) -> None:
        self.env.update_res_state(obj=self, state='WAITING')
        return None
    
    def sim_control(self) -> None:
        while True:
            # initialise state by passivating machines
            # resources are activated by other resources
            if len(self.logic_queue) == 0:
                yield self.passivate()
            logger_prodStations.debug(f"[MACHINE: {self}] is getting job from queue")
            
            # get job function from PARENT CLASS
            # ONLY PROCESSING STATIONS ARE ASKING FOR SEQUENCING
            # state setting --> 'PROCESSING'
            job, job_proc_time = yield from self.get_job()
            # [STATS:ProdStation] count number of inputs
            self.num_inputs += 1
            # [CONTENT:ProdStation] add content
            self.add_content(job=job)
            
            # RELEVANT INFORMATION BEFORE PROCESSING
            #current_op.start_time = self.env.now()
            self.env.dispatcher.update_job_process_info(job=job, preprocess=True)
            #proc_time = job.current_proc_time
            logger_prodStations.debug(f"[START] job ID {job.job_id} at {self.env.now()} on machine ID {self.custom_identifier} \
                with proc time {job_proc_time}")
            # PROCESSING
            yield self.hold(job_proc_time)
            # RELEVANT INFORMATION AFTER PROCESSING
            self.env.dispatcher.update_job_process_info(job=job, preprocess=False)
            
            logger_prodStations.debug(f"[END] job ID {job.job_id} at {self.env.now()} on machine ID {self.custom_identifier}")
            # only place job if there are open operations left
            # maybe add to 'put_job' method
            target_proc_station = yield from self.put_job(job=job)
            # [CONTENT:ProdStation] remove content
            self.remove_content(job=job)
            
    def post_process(self) -> None:
        return None
    
    def finalise(self) -> None:
        """
        method to be called at the end of the simulation run by 
        the environment's "finalise_sim" method
        """
        # each resource object class has dedicated finalise methods which 
        # must be called by children
        super().finalise()

<a id='machine'></a>

In [727]:
class Machine(ProcessingStation):
    
    def __init__(
        self,
        resource_type: str = 'Machine',
        **kwargs,
    ) -> None:
        """
        env:        SimPy Environment in which machine is embedded
        num_slots:  capacity of the machine, if multiple processing 
                    slots available at the same time > 1, default=1
        """
        # assert object information
        self.res_type = resource_type
        
        # intialize base class
        super().__init__(**kwargs)
        
    

<a id='buffer'></a>

In [728]:
class Buffer(sim.Store, InfrastructureObject):
    
    def __init__(
        self,
        capacity: float,
        resource_type: str = 'Buffer',
        possible_states: Iterable[str] = (
            'INIT',
            'FINISH',
            'TEMP',
            'FULL',
            'EMPTY',
            'INTERMEDIATE',
            'FAILED',
            'PAUSED',
        ),
        fill_level: int = 0,
        **kwargs,
    ) -> None:
        """
        capacity: capacity of the buffer, can be infinite
        """
        # assert object information
        self.res_type = resource_type
        self.start_fill_level = fill_level
        
        # intialize base classes
        # using hard-coded classes because salabim does not provide 
        # interfaces for multiple inheritance
        sim.Store.__init__(self, capacity=capacity, env=env)
        InfrastructureObject.__init__(
                            self, capacity=capacity, 
                            possible_states=possible_states, **kwargs)
        
        # material flow relationships
        self._associated_prod_stations: set[ProcessingStation] = set()
        self._count_associated_prod_stations: int = 0
    
    @property
    def level_db(self) -> DataFrame:
        return self._stat_monitor.level_db
    
    @property
    def wei_avg_fill_level(self) -> float:
        return self._stat_monitor.wei_avg_fill_level
    
    
    ### MATERIAL FLOW RELATIONSHIP
    def add_prod_station(
        self,
        prod_station: ProcessingStation
    ) -> None:
        """
        function to add processing stations which are associated with 
        """
        if not isinstance(prod_station, ProcessingStation):
            raise TypeError(f"Object is no ProcessingStation type. Only objects of type ProcessingStation can be added to a buffer.")
        
        # check if adding a new resource exceeds the given capacity
        # each associated processing station needs one storage place in the buffer
        # else deadlocks are possible
        if (self._count_associated_prod_stations + 1) > self.cap:
            raise UserWarning(f"Tried to add a new resource to buffer {self}, but the number of associated \
                resources exceeds its capacity which could result in deadlocks.")
        
        # check if processing station can be added
        if prod_station not in self._associated_prod_stations:
            self._associated_prod_stations.add(prod_station)
            self._count_associated_prod_stations += 1
        else:
            logger_buffers.warning(f"The Processing Station >>{prod_station}<< is already associated with the resource >>{self}<<. \
                Processing Station was not added to the resource.")
        
    def remove_prod_station(
        self,
        prod_station: ProcessingStation
    ) -> None:
        """
        removing a processing station from the current associated ones
        """
        if prod_station in self._associated_prod_stations:
            self._associated_prod_stations.remove(prod_station)
            self._count_associated_prod_stations -= 1
        else:
            raise KeyError(f"The processing station >>{prod_station}<< is not associated with the resource >>{self}<< and \
                therefore could not be removed.")
    
    ### PROCESS LOGIC
    def pre_process(self) -> None:
        self.env.update_res_state(obj=self, state='EMPTY')
        return None
    
    def sim_control(self) -> None:
        while True:
            logger_prodStations.debug(f"[BUFFER: {self}] Invoking at {self.env.now()}")
            # full
            if self.available_quantity() == 0:
                # [STATE] FULL
                self.env.update_res_state(obj=self, state='FULL')
                logger_prodStations.debug(f"[BUFFER: {self}] Set to 'FULL' at {self.env.now()}")
            # empty
            elif self.available_quantity() == self.capacity():
                # [STATE] EMPTY
                self.env.update_res_state(obj=self, state='EMPTY')
                logger_prodStations.debug(f"[BUFFER: {self}] Set to 'EMPTY' at {self.env.now()}")
            else:
                # [STATE] INTERMEDIATE
                self.env.update_res_state(obj=self, state='INTERMEDIATE')
                logger_prodStations.debug(f"[BUFFER: {self}] Neither 'EMPTY' nor 'FULL' at {self.env.now()}")
            
            yield self.passivate()
        
        return None
            
    def post_process(self) -> None:
        return None
    
    def finalise(self) -> None:
        """
        method to be called at the end of the simulation run by 
        the environment's "finalise_sim" method
        """
        # each resource object class has dedicated finalise methods which 
        # must be called by children
        super().finalise()

[Jump to top](#top)

In [729]:
random.normalvariate(10, 2)

10.490652683415727

**Sources:**
- entity generation:
    - constant
    - random

<a id='source'></a>

In [791]:
class Source(InfrastructureObject):
    
    def __init__(
        self,
        resource_type: str = 'Source',
        proc_time: float = 1.,
        random_generation: bool = False,
        job_generator: RandomJobGenerator | None = None,
        num_gen_jobs: int = 5,
        **kwargs,
    ) -> None:
        """
        num_gen_jobs: total number of jobs to be generated
        """
        # assert object information and register object in the environment
        self.res_type = resource_type
        
        # random generation
        if random_generation and job_generator is None:
            raise ValueError("Random job generator instance needed for random job generation")
        
        self.random_generation = random_generation
        self.job_generator = job_generator
        
        ### REWORK
        # initialize component with necessary process function
        random.seed(42)
        super().__init__(**kwargs)
        
        # parameters
        self.proc_time = proc_time
        self.num_gen_jobs = num_gen_jobs
    
    def _obtain_proc_time(self) -> float:
        """
        function to generate a constant or random processing time
        """
        if self.random_generation:
            # random generation, add later
            return self.proc_time
        else:
            return self.proc_time
    
    ### PROCESS LOGIC
    def pre_process(self) -> None:
        self.env.update_res_state(obj=self, state='PROCESSING')
        return None
    
    def sim_control(self) -> None:
        # id counter for debugging, else endless generation
        count = 0
        dispatcher = self.env.dispatcher
        
        # use machine custom identifiers for generation
        machines = self.env.res_db.loc[self.env.res_db['res_type']=='Machine']
        machines_custom_ids = machines['custom_id'].to_list()
        
        # use station group custom identifiers for generation
        station_groups_custom_ids = self.env.station_group_db['custom_group_id'].to_list()
        
        while count < self.num_gen_jobs:
            # start at t=0 with generation
            # generate object
            ## random job properties
            ## currently: each job passes each machine, only one machine of each operation type
            #mat_ProcTimes, mat_JobMachID = self.job_generator.gen_rnd_job(n_machines=self.env.num_proc_stations)
            #job = Job(dispatcher=dispatcher, proc_times=mat_ProcTimes.tolist(), 
            #          machine_order=mat_JobMachID.tolist())
            #mat_ProcTimes, mat_JobMachID = self.job_generator.gen_rnd_job_by_ids(ids=machines_custom_ids)
            mat_ProcTimes, mat_JobExOrder = self.job_generator.gen_rnd_job_by_ids(ids=station_groups_custom_ids, min_proc_time=5)
            job = Job(dispatcher=dispatcher, proc_times=mat_ProcTimes, 
                      station_group_ex_order=mat_JobExOrder)
            # [Call:DISPATCHER]
            dispatcher.release_job(job=job)
            # [STATS:Source] count number of inputs (source: generation of jobs or entry in pipeline)
            # implemented in 'get_job' method which is not executed by source objects
            self.num_inputs += 1
            logger_sources.debug(f"[SOURCE: {self}] Generated {job} at {self.env.now()}")
            
            logger_sources.debug(f"[SOURCE: {self}] Request allocation...")
            # put job via 'put_job' function, implemented in parent class 'InfrastructureObject'
            target_proc_station = yield from self.put_job(job=job)
            logger_sources.debug(f"[SOURCE: {self}] PUT JOB with ret = {target_proc_station}")
            # [STATE:Source] put in 'WAITING' by 'put_job' method but still processing
            # only 'WAITING' if all jobs are generated
            self.env.update_res_state(obj=self, state='PROCESSING')
            
            # hold for defined generation time (constant or statistically distributed)
            # if hold time elapsed start new generation
            proc_time = self._obtain_proc_time()
            logger_sources.debug(f"[SOURCE: {self}] Hold for >>{proc_time}<< at {self.env.now()}")
            yield self.hold(proc_time)
            # set counter up
            count += 1
        
        # [STATE:Source] WAITING
        self.env.update_res_state(obj=self, state='WAITING')
        
        return None
            
    def post_process(self) -> None:
        return None
    

In [792]:
class Sink(InfrastructureObject):
    
    def __init__(
        self,
        resource_type: str = 'Sink',
        **kwargs,
    ) -> None:
        """
        num_gen_jobs: total number of jobs to be generated
        """
        # assert object information and register object in the environment
        self.res_type = resource_type
        
        # initialize parent class
        super().__init__(**kwargs)
        
    
    ### STATE SETTING
    
    ### PROCESS LOGIC
    def pre_process(self) -> None:
        # currently sinks are 'PROCESSING' the whole time
        self.env.update_res_state(obj=self, state='PROCESSING')
        return None
    
    def sim_control(self) -> None:
        dispatcher = self.env.dispatcher
        while True:
            # in analogy to ProcessingStations
            if len(self.logic_queue) == 0:
                yield self.passivate()
            
            logger_sinks.debug(f"[SINK: {self}] is getting job from queue")
            # get job, simple FIFO
            job: Job = self.logic_queue.pop()
            # [Call:DISPATCHER] data collection: finalise job
            dispatcher.finish_job(job=job)
            #job.finalise()
            # destroy job object ???
            # if job object destroyed, unsaved information is lost
            # if not destroyed memory usage could increase
            
    def post_process(self) -> None:
        return None
    

In [794]:
env = SimulationEnvironment(name='base')
job_generator = RandomJobGenerator(seed=2)
dispatcher = Dispatcher(env=env, priority_rule='FIFO')
#buffer = Buffer(capacity=10, env=env, custom_identifier=10)
custom_id = 0
# resources
for machine in range(1):
    buffer = Buffer(capacity=20, env=env, custom_identifier=(10+machine))
    
    if machine >= 2:
        custom_id += 1
    
    MachInst = Machine(env=env, custom_identifier=machine, buffers=[buffer],
                       station_group_custom_id=custom_id, station_group_name='Bohrerei')
    
    if machine == 0:
        buffbuff = buffer
        machmach = MachInst
    #MachInst = Machine(env=env, custom_identifier=machine)

source = Source(env=env, custom_identifier='source', proc_time=1, 
                random_generation=True, job_generator=job_generator, num_gen_jobs=5)
sink = Sink(env=env, custom_identifier='sink')


In [795]:
env.check_integrity()

In [796]:
dispatcher.possible_prio_rules()

{'FIFO', 'LIFO', 'LPT', 'SPT'}

In [797]:
dispatcher.possible_alloc_rules()

{'RANDOM', 'UTILISATION', 'WIP_LOAD_JOBS', 'WIP_LOAD_TIME'}

In [798]:
#dispatcher.curr_prio_rule = 'SPT'
dispatcher.curr_alloc_rule = 'WIP_LOAD_TIME'
#dispatcher.curr_alloc_rule = 'UTILISATION'

INFO:dispatcher:Changed allocation rule to WIP_LOAD_TIME


In [799]:
env.res_db

,custom_id,resource,name,res_type,state
env_id,,,,,
1,10,(),Buffer_env_1,Buffer,INIT
2,0,Machine (Machine_env_2),Machine_env_2,Machine,INIT
3,source,Source (Source_env_3),Source_env_3,Source,INIT
4,sink,Sink (Sink_env_4),Sink_env_4,Sink,INIT


In [800]:
env.station_group_db

,custom_group_id,station_group_name,station_group
station_group_id,,,
0,0,Bohrerei,{Machine (Machine_env_2)}


In [801]:
env.name()

'base'

In [802]:
env.run()
env.finalise_sim()

DEBUG:base:----> Process logic of Buffer (Buffer_env_1)
DEBUG:base:----> Process logic of Machine (Machine_env_2)
DEBUG:base:----> Process logic of Source (Source_env_3)
INFO:dispatcher:Successfully registered job with JobID 0 and name J_gen_0
INFO:dispatcher:Successfully registered operation with OpID 0 and name O_gen_0
INFO:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] REQUEST TO DISPATCHER FOR ALLOCATION
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] Available stations at 0.0 are [Machine (Machine_env_2)]
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] WIP LOAD TIME of target_station=Machine (Machine_env_2) is 0.00
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] Next operation is Operation(ProcTime: 6, StationGroupID: 0) with machine group (machine) Machine (Machine_env_2)
DEBUG:base:----> Process logic of Sink (Sink_env_4)
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] REQUEST TO DISPATCHER FOR SEQUENCING
INFO

INFO:dispatcher:Successfully registered job with JobID 3 and name J_gen_3
INFO:dispatcher:Successfully registered operation with OpID 3 and name O_gen_3
INFO:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] REQUEST TO DISPATCHER FOR ALLOCATION
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] Available stations at 3.0 are [Machine (Machine_env_2)]
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] WIP LOAD TIME of target_station=Machine (Machine_env_2) is 18.00
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] Next operation is Operation(ProcTime: 9, StationGroupID: 0) with machine group (machine) Machine (Machine_env_2)
INFO:dispatcher:Successfully registered job with JobID 4 and name J_gen_4
INFO:dispatcher:Successfully registered operation with OpID 4 and name O_gen_4
INFO:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] REQUEST TO DISPATCHER FOR ALLOCATION
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] Available 

In [803]:
dispatcher.cycle_time

35.0

In [804]:
env.res_db

,custom_id,resource,name,res_type,state
env_id,,,,,
1,10,(),Buffer_env_1,Buffer,FINISH
2,0,Machine (Machine_env_2),Machine_env_2,Machine,FINISH
3,source,Source (Source_env_3),Source_env_3,Source,FINISH
4,sink,Sink (Sink_env_4),Sink_env_4,Sink,FINISH


In [805]:
mch = env.get_res_obj_by_prop(val=0, property='custom_id')

In [806]:
m = mch.stat_monitor

In [807]:
m.WIP_time_db

,sim_time,duration,level
0,1.0,1.0,11.0
1,2.0,1.0,18.0
2,3.0,1.0,27.0
3,4.0,1.0,35.0
4,6.0,2.0,29.0
5,11.0,5.0,24.0
6,18.0,7.0,17.0
7,27.0,9.0,8.0
8,35.0,8.0,0.0


In [813]:
m.wei_avg_WIP_level_time

13.142857142857142

In [816]:
ret = m.draw_WIP_level(use_num_jobs_metric=False)

In [809]:
buff = list(mch.buffers)
buff = buff[0]

In [810]:
q = sim.Queue()

In [811]:
fig = buff.stat_monitor.draw_fill_level()

In [698]:
fig = buffbuff.stat_monitor.draw_fill_level()

In [685]:
buffer.wei_avg_fill_level

5.288461538461538

In [686]:
MachInst.custom_identifier

3

In [697]:
buff = machmach.buffers

{Buffer (Buffer_env_1)}

In [704]:
m = mch.stat_monitor

In [705]:
ret = m.draw_WIP_level()

In [695]:
m.WIP_time_db

,sim_time,duration,level
0,104.0,104.0,0.0


In [696]:
fig = machmach.stat_monitor.draw_state_bar_chart()

In [275]:
dispatcher.job_db

,custom_id,job,name,job_type,total_proc_time,creation_date,release_date,entry_date,exit_date,lead_time,state
job_id,,,,,,,,,,,
0,None,Job (J_gen_0),J_gen_0,Job,4.0,0.0,0.0,0.0,4.0,4.0,FINISH
1,None,Job (J_gen_1),J_gen_1,Job,1.0,4.0,4.0,0.0,5.0,1.0,FINISH
2,None,Job (J_gen_2),J_gen_2,Job,4.0,8.0,8.0,0.0,12.0,4.0,FINISH
3,None,Job (J_gen_3),J_gen_3,Job,9.0,12.0,12.0,0.0,21.0,9.0,FINISH
4,None,Job (J_gen_4),J_gen_4,Job,7.0,16.0,16.0,0.0,23.0,7.0,FINISH
5,None,Job (J_gen_5),J_gen_5,Job,1.0,20.0,20.0,0.0,21.0,1.0,FINISH
6,None,Job (J_gen_6),J_gen_6,Job,4.0,24.0,24.0,0.0,28.0,4.0,FINISH
7,None,Job (J_gen_7),J_gen_7,Job,8.0,28.0,28.0,0.0,36.0,8.0,FINISH
8,None,Job (J_gen_8),J_gen_8,Job,9.0,32.0,32.0,0.0,41.0,9.0,FINISH


In [276]:
dispatcher.op_db

,job_id,job_name,custom_id,op,name,station_group,station_group_custom_id,station_group_name,target_station_custom_id,target_station_name,duration,creation_date,release_date,entry_date,exit_date,lead_time,state
op_id,,,,,,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 4, StationGroupID: 0)",O_gen_0,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,1,Machine_env_4,4.0,0.0,0.0,0.0,4.0,4.0,FINISH
1,1,J_gen_1,None,"Operation(ProcTime: 1, StationGroupID: 0)",O_gen_1,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,3,Machine_env_8,1.0,4.0,4.0,4.0,5.0,1.0,FINISH
2,2,J_gen_2,None,"Operation(ProcTime: 4, StationGroupID: 0)",O_gen_2,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,1,Machine_env_4,4.0,8.0,8.0,8.0,12.0,4.0,FINISH
3,3,J_gen_3,None,"Operation(ProcTime: 9, StationGroupID: 0)",O_gen_3,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,3,Machine_env_8,9.0,12.0,12.0,12.0,21.0,9.0,FINISH
4,4,J_gen_4,None,"Operation(ProcTime: 7, StationGroupID: 0)",O_gen_4,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,1,Machine_env_4,7.0,16.0,16.0,16.0,23.0,7.0,FINISH
5,5,J_gen_5,None,"Operation(ProcTime: 1, StationGroupID: 0)",O_gen_5,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,0,Machine_env_2,1.0,20.0,20.0,20.0,21.0,1.0,FINISH
6,6,J_gen_6,None,"Operation(ProcTime: 4, StationGroupID: 0)",O_gen_6,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,1,Machine_env_4,4.0,24.0,24.0,24.0,28.0,4.0,FINISH
7,7,J_gen_7,None,"Operation(ProcTime: 8, StationGroupID: 0)",O_gen_7,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,3,Machine_env_8,8.0,28.0,28.0,28.0,36.0,8.0,FINISH
8,8,J_gen_8,None,"Operation(ProcTime: 9, StationGroupID: 0)",O_gen_8,"{Machine (Machine_env_6), Machine (Machine_env...",0,Bohrerei,1,Machine_env_4,9.0,32.0,32.0,32.0,41.0,9.0,FINISH


In [485]:
job = dispatcher.get_job_obj_by_prop(val=5)

In [486]:
job.stat_monitor.state_durations

,abs [timesteps],rel [%]
BLOCKED,0.0,0.000000
FAILED,0.0,0.000000
PAUSED,0.0,0.000000
PROCESSING,20.0,43.478261
WAITING,26.0,56.521739


In [487]:
op1 = job.operations[0]
#op2 = job.operations[1]

In [488]:
op1.stat_monitor.state_times

{'FINISH': 0.0,
 'WAITING': 8.0,
 'FAILED': 0.0,
 'INIT': 0.0,
 'BLOCKED': 0.0,
 'PAUSED': 0.0,
 'PROCESSING': 6.0,
 'TEMP': 0.0}

In [489]:
job.stat_monitor.state_times

{'FINISH': 0.0,
 'WAITING': 26.0,
 'FAILED': 0.0,
 'INIT': 0.0,
 'BLOCKED': 0.0,
 'PAUSED': 0.0,
 'PROCESSING': 20.0,
 'TEMP': 0.0}

In [282]:
MachInst.custom_identifier

3

In [283]:
env.res_db

,custom_id,resource,name,res_type,state
env_id,,,,,
1,10,(),Buffer_env_1,Buffer,FINISH
2,0,Machine (Machine_env_2),Machine_env_2,Machine,FINISH
3,11,(),Buffer_env_3,Buffer,FINISH
4,1,Machine (Machine_env_4),Machine_env_4,Machine,FINISH
5,12,(),Buffer_env_5,Buffer,FINISH
6,2,Machine (Machine_env_6),Machine_env_6,Machine,FINISH
7,13,(),Buffer_env_7,Buffer,FINISH
8,3,Machine (Machine_env_8),Machine_env_8,Machine,FINISH
9,source,Source (Source_env_9),Source_env_9,Source,FINISH


In [490]:
mach = env.get_res_obj_by_prop(val=0, property='custom_id')
mach.stat_monitor.state_times

{'FINISH': 0.0,
 'WAITING': 64.0,
 'FAILED': 0.0,
 'INIT': 0.0,
 'BLOCKED': 0.0,
 'PAUSED': 0.0,
 'PROCESSING': 40.0,
 'TEMP': 0.0}

In [491]:
buff = env.get_res_obj_by_prop(val=12, property='custom_id')
ret = buff.stat_monitor.draw_fill_level()

In [216]:
ret = buffer.stat_monitor.draw_fill_level()

In [217]:
env.station_group_db

,custom_group_id,station_group_name,station_group
station_group_id,,,
0,0,Bohrerei,"{Machine (Machine_env_8), Machine (Machine_env..."


In [764]:
fig = dispatcher.draw_gantt_chart(use_custom_proc_station_id=True, sort_by_proc_station=True, group_by_station_group=False)

In [497]:
type(fig.data)

tuple

In [357]:
fig = dispatcher.draw_gantt_chart(use_custom_proc_station_id=True, sort_by_proc_station=True)

In [299]:
dispatcher.op_db

,job_id,job_name,custom_id,op,name,station_group,station_group_custom_id,station_group_name,target_station_custom_id,target_station_name,duration,creation_date,release_date,entry_date,exit_date,lead_time,state
op_id,,,,,,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 7, StationGroupID: 0)",O_gen_0,"{Machine (Machine_env_2), Machine (Machine_env...",0,Bohrerei,3,Machine_env_8,7.0,0.0,0.0,0.0,7.0,7.0,FINISH
1,1,J_gen_1,None,"Operation(ProcTime: 1, StationGroupID: 0)",O_gen_1,"{Machine (Machine_env_2), Machine (Machine_env...",0,Bohrerei,1,Machine_env_4,1.0,0.5,0.5,0.5,1.5,1.0,FINISH
2,2,J_gen_2,None,"Operation(ProcTime: 3, StationGroupID: 0)",O_gen_2,"{Machine (Machine_env_2), Machine (Machine_env...",0,Bohrerei,2,Machine_env_6,3.0,1.0,1.0,1.0,4.0,3.0,FINISH
3,3,J_gen_3,None,"Operation(ProcTime: 3, StationGroupID: 0)",O_gen_3,"{Machine (Machine_env_2), Machine (Machine_env...",0,Bohrerei,0,Machine_env_2,3.0,1.5,1.5,1.5,4.5,3.0,FINISH
4,4,J_gen_4,None,"Operation(ProcTime: 8, StationGroupID: 0)",O_gen_4,"{Machine (Machine_env_2), Machine (Machine_env...",0,Bohrerei,1,Machine_env_4,8.0,2.0,2.0,2.0,10.0,8.0,FINISH
5,5,J_gen_5,None,"Operation(ProcTime: 7, StationGroupID: 0)",O_gen_5,"{Machine (Machine_env_2), Machine (Machine_env...",0,Bohrerei,0,Machine_env_2,7.0,2.5,2.5,4.5,11.5,9.0,FINISH


In [300]:
env.res_db

,custom_id,resource,name,res_type,state
env_id,,,,,
1,10,(),Buffer_env_1,Buffer,FINISH
2,0,Machine (Machine_env_2),Machine_env_2,Machine,FINISH
3,11,(),Buffer_env_3,Buffer,FINISH
4,1,Machine (Machine_env_4),Machine_env_4,Machine,FINISH
5,12,(),Buffer_env_5,Buffer,FINISH
6,2,Machine (Machine_env_6),Machine_env_6,Machine,FINISH
7,13,(),Buffer_env_7,Buffer,FINISH
8,3,Machine (Machine_env_8),Machine_env_8,Machine,FINISH
9,source,Source (Source_env_9),Source_env_9,Source,FINISH


In [220]:
env.station_group_db

,custom_group_id,station_group_name,station_group
station_group_id,,,
0,0,Bohrerei,"{Machine (Machine_env_8), Machine (Machine_env..."


In [221]:
stat_gr = env.get_station_group_by_prop(val=0, property='station_group_id')

In [222]:
stat_lst = stat_gr.as_list()

In [223]:
stat_lst

[Machine (Machine_env_8),
 Machine (Machine_env_4),
 Machine (Machine_env_6),
 Machine (Machine_env_2)]

In [224]:
m = stat_lst[1]

In [225]:
m.stat_monitor.state_durations

,abs [timesteps],rel [%]
BLOCKED,0.0,0.000000
FAILED,0.0,0.000000
PAUSED,0.0,0.000000
PROCESSING,23.0,28.395062
WAITING,58.0,71.604938


In [306]:
m.custom_identifier

2

[Jump to top](#top)

---

#### **Load Objects**

*Changes for station groups:*
- operations now contain station identifiers instead of machine identifiers
- operation registration in the dispatcher now assigns station groups rather than specific machines
- allocation request in the dispatcher now chooses between machines in the associated machine group; if there is only one (single-machine case) then there are no parallel machines
- allocation request also returns associated buffers **--> buffer management implemented in Dispatcher**

*Buffers:*

~~3 ways to implement buffers with station groups~~:
1. one buffer for the whole group
1. one buffer for each machine
1. mix case: buffer for each machine plus one station group buffer

**Station groups are accessed to obtain processing stations which havee associated buffers --> nothin changes compared to the current behaviour**

*Job creation:*
- now each job must contain a list of station group identifiers instead of machine identifiers
- for random generation there has to be information about the order of station groups and how many station groups exist
    - currently implemented: each job passes each machine
    - next step: each job passes each machine group, so the random order now consists of a permutation of the station group identifiers instead of the machine identifiers --> information available in the station group database of the environment

<a id='dispatcher'></a>

In [564]:
class Dispatcher(object):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        priority_rule: str = 'FIFO',
        allocation_rule: str = 'RANDOM',
    ) -> None:
        """
        Dispatcher class for given environment (only one dispatcher for each environment)
        - different functions to monitor all jobs in the environment
        - jobs report back their states to the dispatcher
        """
        
        # job data base as simple Pandas DataFrame
        # column data types
        self._job_prop: dict[str, type] = {
            'job_id': int,
            'custom_id': object,
            'job': object,
            'name': str,
            'job_type': str,
            'total_proc_time': float,
            'creation_date': float,
            'release_date': float,
            'entry_date': float,
            'exit_date': float,
            'lead_time': float,
            'state': str,
        }
        self._job_db: DataFrame = pd.DataFrame(columns=list(self._job_prop.keys()))
        self._job_db: DataFrame = self._job_db.astype(self._job_prop)
        self._job_db: DataFrame = self._job_db.set_index('job_id')
        # properties by which a object can be obtained from the job database
        self._job_lookup_props: set[str] = set(['job_id', 'custom_id', 'name'])
        # properties which can be updated after creation
        self._job_update_props: set[str] = set([
            'creation_date',
            'release_date',
            'entry_date',
            'exit_date',
            'lead_time',
            'state',
        ])
        
        # operation data base as simple Pandas DataFrame
        # column data types
        self._op_prop: dict[str, type] = {
            'op_id': int,
            'job_id': int,
            'job_name': str,
            'custom_id': object,
            'op': object,
            'name': str,
            'station_group': object,
            'station_group_custom_id': object,
            'station_group_name': str,
            'target_station_custom_id': object,
            'target_station_name': str,
            'duration': float,
            'creation_date': float,
            'release_date': float,
            'entry_date': float,
            'exit_date': float,
            'lead_time': float,
            'state': str,
        }
        self._op_db: DataFrame = pd.DataFrame(columns=list(self._op_prop.keys()))
        self._op_db: DataFrame = self._op_db.astype(self._op_prop)
        self._op_db: DataFrame = self._op_db.set_index('op_id')
        # properties by which a object can be obtained from the operation database
        self._op_lookup_props: set[str] = set(['op_id', 'job_id', 'custom_id', 'name', 'machine'])
        # properties which can be updated after creation
        self._op_update_props: set[str] = set([
            'target_station_custom_id',
            'target_station_name',
            'creation_date',
            'release_date',
            'entry_date',
            'exit_date',
            'lead_time',
            'state',
        ])
        
        # register in environment and get EnvID
        self._env = env
        self._env_id: EnvID = self._env.register_dispatcher(self)
        
        ########## PERHAPS REWORK ##########
        self._disposable_jobs: dict[int, Job] = dict()
        self.job_pool: OrderedDict[ObjectID, Job] = OrderedDict()
        ####################################
        # managing IDs
        self._id_types = set(['job', 'op'])
        self._job_id_counter: ObjectID = 0
        self._op_id_counter: ObjectID = 0
        
        # priority rules
        self._priority_rules: set[str] = set([
            'FIFO',
            'LIFO',
            'SPT',
            'LPT',
        ])
        # set current priority rule
        if priority_rule not in self._priority_rules:
            raise ValueError(f"Priority rule {priority_rule} unknown. Must be one of {self._priority_rules}")
        else:
            self._curr_prio_rule = priority_rule
            
        # allocation rule
        self._allocation_rules: set[str] = set([
            'RANDOM',
            'UTILISATION',
            'WIP_LOAD_TIME',
            'WIP_LOAD_JOBS',
        ])
        # set current allocation rule
        if allocation_rule not in self._allocation_rules:
            raise ValueError(f"Allocation rule {allocation_rule} unknown. Must be one of {self._allocation_rules}")
        else:
            self._curr_alloc_rule = allocation_rule
            
        # [STATS] cycle time
        self._cycle_time: float = 0.
    
    ### DATA MANAGEMENT
    def __repr__(self) -> str:
        return f"Dispatcher(env: {self.env.name()}, EnvID: {self._env_id})"
    
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    
    @property
    def env(self) -> SimulationEnvironment:
        return self._env
    
    @property
    def curr_prio_rule(self) -> str:
        return self._curr_prio_rule
    
    @curr_prio_rule.setter
    def curr_prio_rule(self, rule) -> None:
        if rule not in self._priority_rules:
            raise ValueError(f"Priority rule {rule} unknown. Must be one of {self._priority_rules}")
        else:
            self._curr_prio_rule = rule
            logger_dispatcher.info(f"Changed priority rule to {rule}")
            
    def possible_prio_rules(self) -> set[str]:
        return self._priority_rules
    
    @property
    def curr_alloc_rule(self) -> str:
        return self._curr_alloc_rule
    
    @curr_alloc_rule.setter
    def curr_alloc_rule(self, rule) -> None:
        if rule not in self._allocation_rules:
            raise ValueError(f"Allocation rule {rule} unknown. Must be one of {self._allocation_rules}")
        else:
            self._curr_alloc_rule = rule
            logger_dispatcher.info(f"Changed allocation rule to {rule}")
            
    def possible_alloc_rules(self) -> set[str]:
        return self._allocation_rules
    
    def _obtain_job_id(self) -> ObjectID:
        """Simple counter function for managing job IDs"""
        # assign id and set counter up
        job_id = self._id_counter
        self._id_counter += 1
        
        return job_id
    
    def _obtain_op_id(self) -> ObjectID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        op_id = self._op_id_counter
        self._op_id_counter += 1
        
        return op_id
    
    def _obtain_load_obj_id(
        self,
        load_type: str,
    ) -> ObjectID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        
        if load_type not in self._id_types:
            raise ValueError(f"Given type {type} not valid. Choose from '{self._id_types}'")
        
        match load_type:
            case 'job':
                ident_no = self._job_id_counter
                self._job_id_counter += 1
            case 'op':
                ident_no = self._op_id_counter
                self._op_id_counter += 1
        
        return ident_no
    
    @property
    def cycle_time(self) -> float:
        return self._cycle_time
    
    def _calc_cycle_time(self) -> None:
        """Obtaining the current cycle time or maximum exit date of all operations

        Returns
        -------
        float
            maximum exit date of all operations at the time of execution
        """
        self._cycle_time = self._op_db['exit_date'].max()
        
        return None
    
    ### JOBS ###
    def register_job(
        self,
        obj: Job,
        custom_identifier: CustomID | None,
        name: str | None,
        state: str,
    ) -> tuple[SimulationEnvironment, ObjectID, str, float]:
        """
        registers an job object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated jobs
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # obtain id
        job_id = self._obtain_load_obj_id(load_type='job')
        
        # custom name
        if name is None:
            name = f'J_gen_{job_id}'
        
        # time of creation
        creation_date = self.env.now()
        
        # new entry for job data base
        new_entry: DataFrame = pd.DataFrame({
                                'job_id': [job_id],
                                'custom_id': [custom_identifier],
                                'job': [obj],
                                'name': [name],
                                'job_type': [obj.job_type],
                                'total_proc_time': [obj.total_proc_time],
                                'creation_date': [creation_date],
                                'release_date': [obj.time_release],
                                'entry_date': [obj.time_entry],
                                'exit_date': [obj.time_exit],
                                'lead_time': [obj.lead_time],
                                'state': [state]})
        new_entry = new_entry.astype(self._job_prop)
        new_entry = new_entry.set_index('job_id')
        self._job_db = pd.concat([self._job_db, new_entry])
        
        logger_dispatcher.info(f"Successfully registered job with JobID {job_id} and name {name}")
        
        return self.env, job_id, name, creation_date
    
    def update_job_db(
        self,
        job: Job,
        property: str,
        val: float | str,
    ) -> None:
        """
        updates the information of a job for a given porperty
        """
        # check if property is a filter criterion
        if property not in self._job_update_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._job_update_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        self._job_db.loc[job.job_id, property] = val
        
        return None
    
    def release_job(
        self,
        job: Job,
    ) -> None:
        """
        used to signal the release of the given job
        necessary for time statistics
        """
        current_time = self.env.now()
        job.time_release = current_time
        job.is_released = True
        self.update_job_db(job=job, property='release_date', val=job.time_release)
        
        return None
    
    def finish_job(
        self,
        job: Job,
    ) -> None:
        """
        used to signal the exit of the given job
        necessary for time statistics
        """
        # [STATS]
        current_time = self.env.now()
        job.time_exit = current_time
        job.is_finished = True
        job.lead_time = job.time_exit - job.time_release
        # update databases
        self.update_job_state(job=job, state='FINISH')
        self.update_job_db(job=job, property='exit_date', val=job.time_exit)
        self.update_job_db(job=job, property='lead_time', val=job.lead_time)
        # [MONITOR] finalise stats
        job.stat_monitor.finalise_stats()
        
        return None
    
    def update_job_process_info(
        self,
        job: Job,
        preprocess: bool,
    ) -> None:
        """
        method to write necessary information of the current operation before and after processing,
        invoked by Infrastructure Objects
        """
        # get current operation of the job instance
        current_op = job.current_op
        # before processing
        if preprocess:
            # operation enters Processing Station
            #self.release_operation(op=current_op)
            self.enter_operation(op=current_op)
            ############# ENTRY OF JOB
            #current_op.start_time = self.env.now()
            return None
        # after processing
        else:
            # finalise current op
            #logger_dispatcher.debug(f"OP {current_op} is finalised")
            self.finish_operation(op=current_op)
            #current_op.finalise()
            job.num_finished_ops += 1
            
            return None
    
    def update_job_state(
        self,
        job: Job,
        state: str,
    ) -> None:
        """method to update the state of a job in the job database"""
        # update state tracking of the job instance
        job.stat_monitor.set_state(state=state)
        # update job database
        self.update_job_db(job=job, property='state', val=state)
        #self._job_db.loc[job.job_id, 'state'] = state
        # only update operation state if it is not finished
        # operations are finished by post-process call to their 'finalise' method
        
        # update state of the corresponding operation
        if job.current_op is not None:
            self.update_operation_state(op=job.current_op, state=state)
        
        return None
    
    ### OPERATIONS ###
    def register_operation(
        self,
        obj: Operation,
        station_group_identifier: CustomID,
        custom_identifier: CustomID | None,
        name: str | None,
        state: str,
    ) -> tuple[ObjectID, str, ProcessingStation, float]:
        """
        registers an operation object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated operations
        
        obj: operation to register
        machine_identifier: custom ID of the associated machine (user interface)
        custom_identifier: custom identifier of the operation 
            (kept for consistency reasons, perhaps remove later)
        name: assigned name the operation
        status: for future features if status of operations is tracked
        
        outputs:
        op_id: assigned operation ID
        name: assigned name
        machine: corresponding machine infrastructure object
        """
        # obtain id
        op_id = self._obtain_load_obj_id(load_type='op')
        # time of creation
        creation_date = self.env.now()
        
        # custom name
        if name is None:
            name = f'O_gen_{op_id}'
        
        # corresponding machine object on which operation is performed
        #machine = self._env.get_res_obj_by_prop(property='custom_id', val=machine_identifier)
        # corresponding machine object on which operation is performed
        station_group = self._env.get_station_group_by_prop(property='custom_group_id', val=station_group_identifier)
        
        # new entry for operation data base
        new_entry: DataFrame = pd.DataFrame({
                                'op_id': [op_id],
                                'job_id': [obj.job_id],
                                'job_name': [obj.job.name()],
                                'custom_id': [custom_identifier],
                                'op': [obj],
                                'name': [name],
                                'station_group': [station_group],
                                'station_group_custom_id': [station_group.custom_identifier],
                                'station_group_name': [station_group.name],
                                'target_station_custom_id': [None],
                                'target_station_name': [None],
                                'duration': [obj.proc_time],
                                'creation_date': [creation_date],
                                'release_date': [obj.time_release],
                                'entry_date': [obj.time_entry],
                                'exit_date': [obj.time_exit],
                                'lead_time': [obj.lead_time],
                                'state': [state]})
        new_entry: DataFrame = new_entry.astype(self._op_prop)
        new_entry = new_entry.set_index('op_id')
        self._op_db = pd.concat([self._op_db, new_entry])
        
        logger_dispatcher.info(f"Successfully registered operation with OpID {op_id} and name {name}")
        
        # return machine object
        return op_id, name, station_group, creation_date
    
    def update_operation_db(
        self,
        op: Operation,
        property: str,
        val: float | str,
    ) -> None:
        """
        updates the information of a job for a given porperty
        """
        # check if property is a filter criterion
        if property not in self._op_update_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._op_update_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        self._op_db.loc[op.op_id, property] = val
        
        return None
    
    def update_operation_state(
        self,
        op: Operation,
        state: str,
    ) -> None:
        """method to update the state of a operation in the operation database"""
        # update state tracking of the operation instance
        op.stat_monitor.set_state(state=state)
        # update operation database
        self.update_operation_db(op=op, property='state', val=state)
        #self._op_db.loc[op.op_id, 'state'] = state
        
        return None

    def release_operation(
        self,
        op: Operation,
        target_station: ProcessingStation,
    ) -> None:
        """
        used to signal the release of the given operation
        necessary for time statistics
        """
        current_time = self.env.now()
        # release time
        op.time_release = current_time
        op.is_released = True
        # update operation database
        # release date
        self.update_operation_db(op=op, property='release_date', val=op.time_release)
        # target station: custom identifier + name
        self.update_operation_db(
                    op=op, property='target_station_custom_id', 
                    val=target_station.custom_identifier)
        self.update_operation_db(
                    op=op, property='target_station_name', 
                    val=target_station.name())
        
        return None
    
    def enter_operation(
        self,
        op: Operation,
    ) -> None:
        """
        used to signal the start of the given operation on a Processing Station
        necessary for time statistics
        """
        current_time = self.env.now()
        # starting time processing
        op.time_entry = current_time
        # update operation database
        self.update_operation_db(op=op, property='entry_date', val=op.time_entry)
        
        return None
     
    def finish_operation(
        self,
        op: Operation,
    ) -> None:
        """
        used to signal the finalisation of the given operation
        necessary for time statistics
        """
        current_time = self.env.now()
        # [STATE] finished
        op.is_finished = True
        # [STATS] end + lead time
        op.time_exit = current_time
        op.lead_time = op.time_exit - op.time_release
        
        # update databases
        logger_dispatcher.debug(f"Update databases for OP {op} ID {op.op_id} with [{op.time_exit, op.lead_time}]")
        self.update_operation_state(op=op, state='FINISH')
        self.update_operation_db(op=op, property='exit_date', val=op.time_exit)
        self.update_operation_db(op=op, property='lead_time', val=op.lead_time)
        
        # [MONITOR] finalise stats
        op.stat_monitor.finalise_stats()
        
        return None
    
    ### PROPERTIES ###
    @property
    def job_db(self) -> DataFrame:
        """
        obtain a current overview of registered jobs in the environment
        """
        return self._job_db
    
    @property
    def op_db(self) -> DataFrame:
        """
        obtain a current overview of registered operations in the environment
        """
        return self._op_db

    #@lru_cache(maxsize=200)
    def get_job_obj_by_prop(
        self, 
        val: EnvID | CustomID | str,
        property: str = 'job_id',
        target_prop: str = 'job',
    ) -> Job:
        """
        obtain a job object from the dispatcher by its property and corresponding value
        properties: job_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._job_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._job_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        if property == 'job_id':
            # direct indexing for ID property; job_id always unique, no need for duplicate check
            try:
                temp1: Job = self._job_db.loc[val, target_prop]
                return temp1
            except KeyError:
                raise IndexError(f"There were no jobs found for the property '{property}' \
                                with the value '{val}'")
        else:
            temp1: Series = self._job_db.loc[self._job_db[property] == val, target_prop]
            # check for empty search result, at least one result necessary
            if len(temp1) == 0:
                raise IndexError(f"There were no jobs found for the property '{property}' \
                                with the value '{val}'")
            # check for multiple entries with same prop-value pair
            ########### PERHAPS CHANGE NECESSARY
            ### multiple entries but only one returned --> prone to errors
            elif len(temp1) > 1:
                # warn user
                logger_dispatcher.warning(f"CAUTION: There are multiple jobs which share the \
                            same value '{val}' for the property '{property}'. \
                            Only the first entry is returned.")
            
            return temp1.iat[0]
    
    ### ROUTING LOGIC ###
    def request_job_allocation(
        self,
        job: Job,
    ) -> InfrastructureObject:
        """
        request an allocation decision for the given job 
        (determine the next processing station on which the job shall be placed)
        
        1. obtaining the target station group
        2. select from target station group (e.g. calling RL agent for that group)
        3. return target station (InfrastructureObject)
        
        requester: output side infrastructure object
        request for: infrastructure object instance
        """
        # SIGNALING ALLOCATION DECISION
        # (ONLY IF PARALLEL PROCESSING STATIONS EXIST)
        ## theoretically: obtaining next operation --> information about machine group -->
        ## based on machine group: choice of corresponding allocation agent -->
        ## preparing feature vectors as input --> trigger agent decision -->
        ## map decision to processing station
        
        logger_dispatcher.info(f"[DISPATCHER: {self}] REQUEST TO DISPATCHER FOR ALLOCATION")
        
        # get the next operation of the job
        next_op = job.get_next_operation()
        if next_op is not None:
            # select target station from the operation's station group
            target_station = self._choose_target_station_from_group(
                                            station_group=next_op.target_station_group)
            # with allocation request operation is released
            self.release_operation(op=next_op, target_station=target_station)
        # all operations done, look for sinks
        else:
            sinks = list(self.env.sinks)
            # [PERHAPS CHANGE IN FUTURE]
            # use first sink of the registered ones
            target_station = sinks[0]
        
        logger_dispatcher.debug(f"[DISPATCHER: {self}] Next operation is {next_op} with machine group (machine) {target_station}")
        
        return target_station
    
    def _choose_target_station_from_group(
        self,
        station_group: StationGroup,
    ) -> ProcessingStation:
        """Choosing a target station from a given Station Group

        Parameters
        ----------
        station_group : StationGroup
            station group from which the target station should be obtained

        Returns
        -------
        target_station: ProcessingStation
            station object on which the job should be placed
        """
        # [KPIs] calculate necessary information for decision making
        # put all associated processing stations of that group in 'TEMP' state
        self.env.res_objs_temp_state(res_objs=station_group, reset_temp=False)
        
        # stations in list
        stations = station_group.as_list()
        
        # choose only from available processing stations
        candidates = [ps for ps in stations if ps.stat_monitor.is_available]
        # check if there are available processing stations
        # if not: use all stations
        if candidates:
            avail_stations = candidates
        else:
            avail_stations = stations
        
        logger_dispatcher.debug(f"[DISPATCHER: {self}] Available stations at {self.env.now()} are {avail_stations}")
        
        # apply different strategies to select a station out of the station group
        match self._curr_alloc_rule:
            case 'RANDOM':
                # [RANDOM CHOICE]
                target_station: ProcessingStation = random.choice(avail_stations)
            case 'UTILISATION':
                # [UTILISATION]
                # choose the station with the lowest utilisation to time
                temp = sorted(avail_stations, key=attrgetter('stat_monitor.utilisation'), reverse=True)
                target_station: ProcessingStation = temp.pop()
                logger_dispatcher.debug(f"[DISPATCHER: {self}] Utilisation of {target_station=} is {target_station.stat_monitor.utilisation:.4f}")
            case 'WIP_LOAD_TIME':
                # WIP as load/processing time, choose station with lowest WIP
                temp = sorted(avail_stations, key=attrgetter('stat_monitor.WIP_load_time'), reverse=True)
                target_station: ProcessingStation = temp.pop()
                logger_dispatcher.debug(f"[DISPATCHER: {self}] WIP LOAD TIME of {target_station=} is {target_station.stat_monitor.WIP_load_time:.2f}")
            case 'WIP_LOAD_JOBS':
                # WIP as number of associated jobs, choose station with lowest WIP
                temp = sorted(avail_stations, key=attrgetter('stat_monitor.WIP_load_num_jobs'), reverse=True)
                target_station: ProcessingStation = temp.pop()
                logger_dispatcher.debug(f"[DISPATCHER: {self}] WIP LOAD NUM JOBS of {target_station=} is {target_station.stat_monitor.WIP_load_time:.2f}")
        
        # [KPIs] reset all associated processing stations of that group to their original state
        self.env.res_objs_temp_state(res_objs=station_group, reset_temp=True)
        
        return target_station
    
    def request_job_sequencing(
        self,
        req_obj: ProcessingStation
    ) -> tuple[Job, float]:
        """
        request a sequencing decision for a given queue of the requesting resource
        requester: input side processing stations
        request for: job instance
        
        req_obj: requesting object (ProcessingStation)
        """
        # SIGNALING SEQUENCING DECISION
        # (ONLY IF MULTIPLE JOBS IN THE QUEUE EXIST)
        ## theoretically: get logic queue of requesting object --> information about feasible jobs -->
        ## [*] choice of sequencing agent (based on which properties?) --> preparing feature vector as input -->
        ## trigger agent decision --> map decision to feasible jobs
        ## [*] use implemented priority rules as intermediate step
        
        logger_dispatcher.debug(f"[DISPATCHER: {self}] REQUEST TO DISPATCHER FOR SEQUENCING")
        
        # get logic queue of requesting object
        # contains all feasible jobs for this resource
        logic_queue = req_obj.logic_queue
        # get job from logic queue with currently defined priority rule
        job = self.seq_priority_rule(queue=logic_queue)
        
        return job, job.current_proc_time
    
    def seq_priority_rule(
        self,
        queue: Queue,
    ) -> Job:
        """apply priority rules to a pool of jobs"""
        match self._curr_prio_rule:
            case 'FIFO':
                # salabim queue pops first entry if no index is specified, 
                # not last like in Python
                job = queue.pop()
            case 'LIFO':
                # salabim queue pops first entry if no index is specified, 
                # not last like in Python
                job = queue.pop(-1)
            case 'SPT':
                # sort descending and pop last item
                temp = queue.as_list()
                temp = sorted(temp, key=attrgetter('current_proc_time'), reverse=True)
                job: Job = temp.pop()
                # remove job from original queue
                queue.remove(job)
            case 'LPT':
                # sort ascending and pop last item
                temp = queue.as_list()
                temp = sorted(temp, key=attrgetter('current_proc_time'), reverse=False)
                job: Job = temp.pop()
                # remove job from original queue
                queue.remove(job)
                
        return job
    
    ### ANALYSE ###
    def draw_gantt_chart(
        self,
        use_custom_proc_station_id: bool = True,
        sort_by_proc_station: bool = False,
        sort_ascending: bool = True,
        group_by_station_group: bool = False,
        save_img: bool = False,
        save_html: bool = False,
        file_name: str = 'gantt_chart',
    ) -> PlotlyFigure:
        """
        draw a Gantt chart based on the dispatcher's operation database
        use_custom_machine_id: whether to use the custom IDs of the processing station (True) or its name (False)
        sort_by_proc_station: whether to sort by processing station property (True) or by job name (False) \
            default: False
        sort_ascending: whether to sort in ascending (True) or descending order (False) \
            default: True
        use_duration: plot each operation with its scheduled duration instead of the delta time \
            between start and end; if there were no interruptions both methods return the same results \
            default: False
        """
        # filter operation DB for relevant information
        filter_items: list[str] = [
            'job_name',
            'target_station_custom_id',
            'target_station_name',
            'station_group',
            'station_group_custom_id',
            'entry_date',
            'exit_date',
        ]
        
        df = self._op_db.filter(items=filter_items)
        # calculate delta time between start and end
        df['delta'] = df['exit_date'] - df['entry_date']
        
        # sorting
        sort_key: str = ''
        # chose relevant processing station property
        proc_station_prop: str = ''
        if use_custom_proc_station_id:
            proc_station_prop = 'target_station_custom_id'
        else:
            proc_station_prop = 'target_station_name'
        
        # check if sorting by processing station is wanted and custom ID should be used or not
        if sort_by_proc_station:
            sort_key = proc_station_prop
        else:
            sort_key = 'job_name' 
        
        df = df.sort_values(by=sort_key, ascending=sort_ascending, kind='stable')
        
        # group by value
        if group_by_station_group:
            group_by_key = 'station_group_custom_id'
        else:
            group_by_key = 'job_name'
        
        # build Gantt chart with Plotly Timeline
        fig: PlotlyFigure = px.timeline(df, x_start='entry_date', x_end='exit_date', 
                          y=proc_station_prop, color=group_by_key)
        fig.update_yaxes(type='category', autorange='reversed')
        fig.update_xaxes(type='linear')

        # reset axis scale for every figure element
        # https://stackoverflow.com/questions/66078893/plotly-express-timeline-for-gantt-chart-with-integer-xaxis
        for d in fig.data:
            try:
                # convert to integer if property is of that type in the database
                filt_val = int(d.name)
            except ValueError:
                filt_val = d.name
            filt = df[group_by_key] == filt_val
            d.x = df.loc[filt, 'delta']

        fig.show()
        
        if save_html:
            file = f'{file_name}.html'
            fig.write_html(file)
        
        if save_img:
            file = f'{file_name}.svg'
            fig.write_image(file)
        
        return fig
    
    ### DISPOSABLE JOBS
    ### STILL NECESSARY???
    def add_disposable_job(
        self,
        job: Job,
    ) -> None:
        """
        add job to the disposable ones
        """
        self._disposable_jobs[job.job_id] = job
    
    @property
    def disposable_jobs(self) -> dict[int, Job]:
        return self._disposable_jobs
    
    ################# REWORK ##################
    ### maybe add a corresponding property in the job DB
    def get_disposable_jobs(
        self,
        job_set: OrderedDict,
    ) -> tuple[list[ObjectID], list[Job]]:
        """
        function needs to be reworked, jobs should report back information to a dispatcher instance
        (bottom-up instead of top-down)
        """
        #########################################
        self._disposable_jobs_ID: list[int] = list()
        self._disposable_jobs: list[Job] = list()
        
        for job_id, job in job_set.items():
            if job.is_disposable:
                self._disposable_jobs_ID.append(job_id)
                self._disposable_jobs.append(job)
                
        return self._disposable_jobs_ID, self._disposable_jobs
    
    def finalise(self) -> None:
        """
        method to be called at the end of the simulation run by 
        the environment's "finalise_sim" method
        """
        self._calc_cycle_time()

[Jump to top](#top)

- change ``machine_identifier`` to ``station_group_identifier``
- ``target_machine`` to ``target_station_group``

<a id='operation'></a>

In [565]:
class Operation(object):
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        job: Job,
        proc_time: float,
        station_group_identifier: CustomID,
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        state: str = 'INIT',
        possible_states: Iterable[str] = (
            'INIT',
            'FINISH',
            'WAITING', 
            'PROCESSING', 
            'BLOCKED', 
            'FAILED', 
            'PAUSED',
        ),
        **kwargs,
    ) -> None:
        """
        identifier: operation's ID
        proc_times: operation's processing times
        machine_identifier: ID of machine on which operation is processed
        """
        # !!!!!!!!! perhaps processing times in future multiple entries depending on associated machine
        # change of input format necessary, currently only one machine for each operation
        # no groups, no differing processing times for different machines 
        # initialise parent class if available
        super().__init__(**kwargs)
        
        # assert operation information
        self._dispatcher = dispatcher
        self._job = job
        self._job_id = job.job_id
        self._station_group_identifier = station_group_identifier
        
        # [STATS] Monitoring
        self._stat_monitor = Monitor(env=self._dispatcher.env, obj=self, init_state=state, 
                                possible_states=possible_states, **kwargs)
        
        # process information
        # [STATS]
        # time characteristics
        self.proc_time: float = proc_time
        self.lead_time: float = 0.
        # inter-process time characteristics
        # time of release
        self.time_release: float = 0.
        # time of first operation starting point
        self.time_entry: float = 0.
        # time of last operation ending point
        self.time_exit: float = 0.
        # lead time
        self.lead_time: float = 0.
        # starting and end points
        # in future setting starting points in advance possible
        self.is_finished: bool = False
        self.is_released: bool = False
        
        ########### adding machine instances
        ### perhaps adding machine sets if multiple machines possible (machine groups)
        # assignment of machine instance by dispatcher
        # from dispatcher: op_id, name, target_machine
        # register operation instance
        current_state = self._stat_monitor.get_current_state()
        (self._op_id, self.name,
         self.target_station_group,
         self.time_creation) = self.dispatcher.register_operation(
                                                        obj=self, 
                                                        station_group_identifier=self._station_group_identifier,
                                                        custom_identifier=custom_identifier, name=name, 
                                                        state=current_state)
        
    def __repr__(self) -> str:
        return f"Operation(ProcTime: {self.proc_time}, StationGroupID: {self._station_group_identifier})"    
    
    @property   
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def stat_monitor(self) -> Monitor:
        return self._stat_monitor
    
    @property
    def op_id(self) -> ObjectID:
        return self._op_id
    
    @property
    def job(self) -> Job:
        return self._job
    
    @property
    def job_id(self) -> ObjectID:
        return self._job_id

[Jump to top](#top)

<a id='job'></a>

In [566]:
class Job(sim.Component):
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        proc_times: list[float],
        station_group_ex_order: list[CustomID],
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        state: str = 'INIT',
        possible_states: Iterable[str] = (
            'INIT',
            'FINISH',
            'WAITING', 
            'PROCESSING', 
            'BLOCKED', 
            'FAILED', 
            'PAUSED',
        ),
        **kwargs,
    ) -> None:
        """
        ############## ADD DESCRIPTION
        """
        ### BASIC INFORMATION ###
        # assert job information
        self.custom_identifier = custom_identifier
        self.job_type: str = 'Job'
        self._dispatcher = dispatcher
        # sum of the proc times of each operation
        self.total_proc_time: float = sum(proc_times)
        
        # inter-process job state parameters
        # first operation scheduled --> released job
        self.is_released: bool = False
        # job's next operation is disposable
        # true for each new job, maybe reworked in future for jobs with
        # a start date later than creation date
        self.is_disposable: bool = True
        # add job to disposable ones
        #ret = self.dispatcher.add_disposable_job(self)
        # last operation ended --> finished job
        self.is_finished: bool = False
        
        # inter-process time characteristics
        # time of release
        self.time_release: float = 0.
        # time of first operation starting point
        self.time_entry: float = 0.
        # time of last operation ending point
        self.time_exit: float = 0.
        # lead time
        self.lead_time: float = 0.
        
        # current resource location
        self._current_resource: InfrastructureObject | None = None
        
        # [STATS] Monitoring
        self._stat_monitor = Monitor(env=self._dispatcher.env, obj=self, init_state=state, 
                                possible_states=possible_states, **kwargs)
        
        # register job instance
        current_state = self._stat_monitor.get_current_state()
        env, self._job_id, name, self.time_creation = self._dispatcher.register_job(
                                                        obj=self, custom_identifier=self.custom_identifier,
                                                        name=name, state=current_state)
        
        # intialize base class
        super().__init__(env=env, name=name, process='', **kwargs)
        
        ### OPERATIONS ##
        self.operations: deque[Operation] = deque()
        
        for idx, op_proc_time in enumerate(proc_times):
            op = Operation(
                dispatcher=self._dispatcher,
                job=self,
                proc_time=op_proc_time,
                station_group_identifier=station_group_ex_order[idx],
            )
            self.operations.append(op)
            
        self.open_operations = self.operations.copy()
        self.total_num_ops: int = len(self.operations)
        self.num_finished_ops: int = 0
        # current and last OP: properties set by function "get_next_operation"
        self._last_op: Operation | None = None
        self._last_proc_time: float | None = None
        self._current_op: Operation | None = None
        self._current_proc_time: float | None = None
        # rank-like property, set if job enters the infrastructure object
        # acts like a counter to allow easy sorting even if queue order is not maintained
        self._obj_entry_idx: int | None = None
    
    @property
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def stat_monitor(self) -> Monitor:
        return self._stat_monitor
    
    @property
    def job_id(self) -> ObjectID:
        return self._job_id
    
    @property
    def last_op(self) -> Operation | None:
        return self._last_op

    @property
    def last_proc_time(self) -> float | None:
        return self._last_proc_time
    
    @property
    def current_op(self) -> Operation | None:
        """
        returns the current operation of the job
        If a job is currently being processed its current operation is 
        not changed until this operation is finished.
        """
        return self._current_op
    
    @property
    def current_proc_time(self) -> float | None:
        """
        returns the processing time of the current operation
        If a job is currently being processed its current processing time is 
        not changed until this operation is finished.
        """
        return self._current_proc_time
    
    @property
    def obj_entry_idx(self) -> int | None:
        """
        returns the entry index which is set by each infrastructure object
        """
        return self._obj_entry_idx
    
    @property
    def current_resource(self) -> InfrastructureObject | None:
        """
        returns the current resource on which the job lies
        """
        return self._current_resource

    @current_resource.setter
    def current_resource(
        self,
        obj: InfrastructureObject
    ) -> None:
        """setting the current resource object which must be of type InfrastructureObject"""
        if not isinstance(obj, InfrastructureObject):
            raise TypeError(f"From {self}: Object >>{obj}<< muste be of type 'InfrastructureObject'")
        else:
            self._current_resource = obj
    
    def get_next_operation(self) -> Operation | None:
        """
        get next operation
        """
        # last operation information
        self._last_op = self._current_op
        self._last_proc_time = self._current_proc_time
        # current operation information
        if self.open_operations:
            op = self.open_operations.popleft()
            self._current_proc_time = op.proc_time
        else:
            op = None
            self._current_proc_time = None
        
        self._current_op = op
        
        return op
    
    def has_job_id(
        self,
        job_id: ObjectID,
    ) -> bool:
        """
        checks whether the current job has the given id
        """
        if self._job_id == job_id:
            return True
        else:
            return False

[Jump to top](#top)

**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)

---
<a id="MachineBreakdownTest"></a>

# Check interruption

In [567]:
env = sim.Environment()

In [568]:
class TestMachine(sim.Component):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.proc_time = 8.
        
    def process(self):
        yield self.passivate()
        
        print(f"Processing time machine = {self.proc_time}; start processing at {self.env.now()}")
        
        yield self.hold(self.proc_time)
        
        print(f"Processing finished at {self.env.now()}")
        
        
class Interruptor(sim.Component):
    
    def __init__(self, machine, **kwargs):
        super().__init__(**kwargs)
        
        self.machine = machine
        self.time_till_failure = 2.
        self.breakdown_time = 3.
        
    def process(self):
        machine = self.machine
        machine.activate()
        
        yield self.hold(self.time_till_failure)
        
        print(f"Interrupt machine at {self.env.now()} for time units {self.breakdown_time}")
        machine.interrupt()
        
        yield self.hold(self.breakdown_time)
        
        print(f"Resume machine at {self.env.now()}")
        machine.resume()
        
        yield self.passivate()
        
        

In [569]:
machine = TestMachine(env=env)
failure = Interruptor(machine=machine, env=env)

In [570]:
env.run()

Processing time machine = 8.0; start processing at 0.0
Interrupt machine at 2.0 for time units 3.0
Resume machine at 5.0
Processing finished at 11.0


In [571]:
machine.status.value_duration('interrupted')

3.0

In [572]:
machine.status.value_duration('scheduled')

8.0